#**Section 1** - Introduction


This is a report on a **data analysis**, including **regression** and a **neural network** trained on a large preprocessed dataset originally extracted from the music streaming platform *Spotify*.

**The aim is to be able to predict the popularity based on musical data**, and **to reveal and retrieve informative insights into the realtionships between musical data and popularity**.

##**1.1**  Instantiation

###***1.1.1*** - External Library Importation

The following section initialises the necessary libraries and dependencies used throughout this project. Each module serves a specific role in supporting data manipulation, visualisation, machine learning and deep learning.

Identifying module imports. This includes **additional** modules:
> - **numpy** as *np*
> - **random** from ***numpy***
> - **pyplot** from ***matplotlib*** as *plt*
> - **pandas** as *pd*
> - **seaborn** as *sns*
> - **tensorflow** as *tf*

> - **sklearn** - including specifically:
>> - ***OneHotEncoder*** from *preprocessing*,
>> - ***train_test_split*** from *model_selection*,
>> - ***metrics*** as *mt*,
>> - ***LinearRegression*** from *linear_model*
>> - and ***RandomForestRegressor*** from *ensemble*

> - **xgboost** as *xgb*

> - **tqdm** from *tqdm*

In [ ]:
import numpy as np
from numpy import random
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn import metrics as mt
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

from tqdm import tqdm   #Progress bar
#if loading bars are not working properly, change to this version:
# from tqdm.notebook import tqdm

> **NumPy** - numerical array operations and randomness for reproducibility

> **Pandas** - efficient tabular data storage, manipulation and cleaning

> **Matplotlib and Seaborn** - essential tools for static and interactive plotting

> **TensorFlow** - enables construction, training and deployment of deep neural networks

> **scikit-learn** - for classical machine learning, including regression and data splitting

> **XGBoost** - an advanced gradient-boosting framework highly effective for structured data

> **TQDM** - provides loading bars for any iterable, useful during long computations

The combination of these libraries forms the foundation of the analysis workflow, especially given the scale of the dataset (~1,000,000 entries), and the multistage processing pipeline required for this prediction task.


.

.

.

.

.

.

.



This sets our results up to be more interpretable when using **Pandas**:

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

#https://stackoverflow.com/questions/55394854/how-to-change-the-format-of-describe-output


.

.

.

.

.

.

.



### ***1.1.2*** - Data Extraction


The raw dataset consists of approximately one million songs originally sourced from Spotify's API, and then sourced from *Kaggle* - https://www.kaggle.com/datasets/amitanshjoshi/spotify-1million-tracks/code. Each entry includes both numerical audio features (e.g. danceability, energy, loudness, valence) and categorical metadata (e.g. artist name, genre, release year). The dataset also includes the song's popularity score, which serves as the primary target variable for prediction throughout this project.

**ENSURE TO LOAD 'spotidata.csv' (the *spotify-1million-tracks* dataset) INTO THE CURRENT WORK SPACE OF THE NOTEBOOK BEFORE RUNNING.**

In [ ]:
#import the dataset
df = pd.read_csv('spotidata.csv')

**[END]**

.

.

.

.

.

.

.

.

.

.


##**1.2**  Initial Analysis

Before any form of cleaning or transformation, it is important to gain a clear understanding of the raw dataset's structure and contents. This stage acts as a first-pass check for any unusual entries or issues that might affect downstream processing. Exploring the general shape, statistical distribution, and feature completeness can often reveal inconsistencies, outliers or early indicators of correlation.

At this point, we are primarily interested in:

> **Basic structure** - how many rows and features are present, and whether the data types appear appropriate (e.g. numerical for audio features, categorical for genre and key).

> **Distribution of values** - inspecting histograms helps identify skew, clustering or potentially problematic values such as extreme outliers in tempo, loudness or popularity.

> **Missing or null data** - a quick check ensures that we understand what proportion of the data may need to be dropped or imputed later.

> **Coverage across categories** - such as genre and year, as sparse classes or unusual categories may need to be removed.

This early analysis helps guide decisions in the preprocessing stage, where more deliberate shaping of the dataset takes place.

###***1.2.1*** - Basic Structure

Previewing the Dataset and the Structure

In [ ]:
display(df)

print(f"Number of songs: {df.shape[0]}")
print(f"Number of features: {df.shape[1]}")
print("\n=== Data Types ===")
print(df.dtypes)


.

.


In [ ]:
print("\n=== Unnormalised Min. and Max. Values ===\n")
# Get the max row
max_values = df.describe().loc['max']

cols_to_display = max_values > 1

# Display the descriptive statistics DataFrame filtered by the boolean mask
display(df.describe().loc[:, cols_to_display])


.

.

.

.

.

.

.



###***1.2.2*** - Distribution of Values

Summary statistics for numerical columns:

In [ ]:
print("\n=== Descriptive Statistics ===\n")
df.describe()


.

.

.

.

.

.

.



###***1.2.3*** - Missing or null Data

Missing data check:

In [ ]:
print("\n=== Null values per column ===\n")
print(df.isnull().sum())


.

.

.

.

.

.

.



###***1.2.4*** - Coverage across Categories

Printing all unique categories, keys and years that appear in the dataset.

In [ ]:
genres = []
keys = []
years = []


for genre in df['genre'].unique():
  genres.append(genre)

for key in df['key'].unique():
  keys.append(key)

for year in df['year'].unique():
  years.append(year)


print(len(genres),"different Genres in total:",df['genre'].unique(),"\n\n")
print(len(keys),"different Keys in total:",df['key'].unique(),"\n\n")
print(len(years),"different Years in total:",df['year'].unique(),"\n\n")

**[END]**

.

.

.

.

.

.

.

.

.

.


##**1.3**  Data Preprocessing

###***1.3.1*** - Data Cleaning

This section removes any incomplete or redundant entries from the dataset. Null values are dropped to avoid issues during model training, particularly because imputation is not appropriate for our audio-based features in this circumstance. Additionally, The duplicate index column - introduced during CSV export - is also removed to ensure clean column alignment throughout.

In addition to removing null values and redundant index columns, this section also filters out genres that are either underrepresented or unsuitable for generalised modelling and poorly identified time signatures. For genres, this includes edge-case categories (e.g. *children*, *comedy*) and culturally localised genres with insufficient representation. Removing these helps focus the analysis on mainstream and musically consistent content, improving overall model reliability.

**Removing null data:**

In [ ]:
# Drop any rows with null (missing) values
df = df.dropna()

**removing duplicate index column:**

In [ ]:
#removing duplicate index column
df = df.drop(df.columns[0],axis=1)


.

.

.

.

.

.

.



**Removing rows with erroneous time signatures:**

In [ ]:
df = df[df.time_signature != 0]
df = df[df.time_signature != 1]


.

.

.

.

.


### ***1.3.2*** - Feature Engineering

This section introduces new features that capture group-level patterns within the data. By calculating average popularity and song count for each artist, genre and year, we embed contextual information that helps models generalise more effectively. These engineered variables help capture popularity trends not immediately obvious from individual song-level features alone.

In [ ]:
def generate_average_popularity(df, category, popularity, title=""):

  i = 0  # counter to track how many songs in current group
  prevVal = category[0]  # start with the first category value
  total_pop = 0  # running total of popularity for group

  category_avg_popularity = []  # will store per-row average popularity
  category_song_count = []      # will store per-row song count

  for x in range(0, len(category) + 1):  # loop over all rows (+1 to handle last group)

    if x < len(category):  # still inside bounds

      if category[x] == prevVal:  # still in the same group
        total_pop += int(popularity[x])  # add this song's popularity
        i += 1  # increment group size

      else:  # new category group starts

        # fill the previous group’s values for each row
        for z in range(i):
          category_avg_popularity.append(total_pop / i)
          category_song_count.append(i)

        # reset values for the new group
        total_pop = int(popularity[x])
        i = 1

      prevVal = category[x]  # update tracker to new category

    else:  # this is after the final row – need to flush the last group
      for z in range(i):
        category_avg_popularity.append(total_pop / i)
        category_song_count.append(i)

  # assign the calculated values as new columns in the dataframe
  df[f'{title}_avg_popularity'] = category_avg_popularity
  df[f'{title}_song_count'] = category_song_count

  return df



.

.

.


**Introducing *artist_avg_popularity*  into the DataFrame**

In [ ]:
df = df.sort_values(by='artist_name')

In [ ]:
artists = df['artist_name'].to_numpy()
popularity = df['popularity'].to_numpy()

In [ ]:
df = generate_average_popularity(df, artists, popularity, 'artist')


.

.

.

.

.




**Introducing *genre_avg_popularity* into the DataFrame**

In [ ]:
df = df.sort_values(by='genre')
#df.tail()

In [ ]:
genre = df['genre'].to_numpy()
popularity = df['popularity'].to_numpy()

In [ ]:
df = generate_average_popularity(df, genre, popularity, 'genre')


.

.

.

.

.




**Introducing *year_avg_popularity* into the DataFrame**

In [ ]:
df = df.sort_values(by='year')

In [ ]:
year = df['year'].to_numpy()
popularity = df['popularity'].to_numpy()

In [ ]:
df = generate_average_popularity(df, year, popularity, 'year')


.

.

.

.

.

.

.


In [ ]:
columnsToDisplay = ['artist_name', 'popularity', 'year', 'genre', 'artist_avg_popularity',
                    'artist_song_count', 'year_avg_popularity', 'year_song_count',
                    'genre_avg_popularity', 'genre_song_count']

display(df[columnsToDisplay].sort_values(by='year', ascending=False))


.

.

.

.

.

.

.



###***1.3.3*** - Data Shaping

The `duration_ms` column is converted to minutes to improve readability and interpretability. This is particularly useful for plotting and UI presentation later on. The original column is dropped after conversion to avoid redundancy.

**Re-scaling *'duration_ms'* :**

In [ ]:
df['duration_mins'] = df['duration_ms'] / 60000 #becomes duration in minutes
df = df.drop(columns=['duration_ms'])


.

.

.

.

.

.

.



**Setting *'year'* to Integer DataType:**

In [ ]:
df['year'] = df['year'].astype(int)


.

.


**Removing 2023 (Incomplete year may skew results inaccurately):**

In [ ]:
# Copy the dataset
data = df.copy()

# Drop duplicates so only one entry per year remains
data = data.drop_duplicates(subset='year')

# Sort by popularity
data = data.sort_values(by='year_avg_popularity', ascending=False)

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(x='year', y='year_avg_popularity', data=data, hue='year_avg_popularity', palette='viridis')
plt.xticks(rotation=45)
plt.title('Average Popularity by Year')
plt.tight_layout()
plt.show()




.

.


In [ ]:
df = df[df['year'] != 2023]


.

.

.

.

.

.

.



###***1.3.4*** - Data Filtering

This section filters out entries that fall outside the scope of meaningful modelling. First, niche and non-musical genres are removed to reduce noise and improve the generalisability of the dataset. Then, numerical features are clipped to realistic bounds based on observed statistical distributions from the initial analysis. This ensures that extreme outliers do not distort learning or affect model scaling.

Niche, non-musical and geographically or culturally specific genres (e.g. *gospel*, *anime*, *cantopop*) are removed to reduce noise and improve model generalisability. These genres tend to be underrepresented or structurally inconsistent with mainstream trends, limiting their value in training.

Several continuous features are also clipped to realistic bounds based on the distributions observed in **Section 1.2**. This includes notably `loudness` (-60 to 0 dB), `tempo` (50-220 BPM), and `duration_mins` (0-20 mins). Count-based features such as `artist_song_count` and `year_song_count` are similarly clipped to prevent extreme outliers from distorting scale or model behaviour or to provide realistic limits for later use in **Section 5**.


.

.

.



**Clipping unclipped features' decided bounds:**

This helps to ensure that there are no outliers, which can skew the data. Additionally, we need specified bounds determined and placed for normalisation, which is a very important part of data preprocessing.

In [ ]:
#                            min           max
# acousticness               0.000000      0.996000
# danceability               0.000000      0.993000
# energy                     0.000000      1.000000
# instrumentalness           0.000000      1.000000
# liveness                   0.000000      0.700000
# loudness---------------- -47.283000      6.172000  BOUND
# speechiness                0.000000      0.967000
# tempo-------------------   0.000000    249.993000  BOUND
# valence                    0.000000      1.000000
# artist_avg_popularity      0.000000     85.000000
# artist_song_count-------   1.000000   1786.000000  BOUND
# genre_avg_popularity       0.001277     55.772119
# genre_song_count-------- 525.000000  20964.000000  BOUND
# year_avg_popularity       11.955374     33.103448
# year_song_count------- 25736.000000  37295.000000  BOUND
# duration_mins-----------   0.034550     96.347433  BOUND
# popularity                 0.000000    100.000000


.

.

.


The following bounds are applied to constrain feature values to realistic and representative ranges:

- **Loudness**: clipped to-60 dB (min) and 0 dB (max)  
   Reflects the typical range of recorded music, avoiding distortion from extremely quiet or anomalously loud entries.

- **Tempo**: clipped to 50 BPM (min) and 220 BPM (max)  
  Covers the full practical spectrum of commercial music tempo without skewing towards outliers.

- **Duration (minutes)**: clipped to 0 (min) and 20 (max)  
  Removes excessively long tracks that behave differently (e.g. live recordings, classical pieces).

- **Artist Song Count**: clipped between 1 and 3000  
  Prevents disproportionately prolific artists from skewing representation.

- **Genre Song Count**: clipped between 0 and 30,000  
  Normalises influence of dominant genres without fully discarding them.

- **Year Song Count**: clipped between 0 and 50,000  
  Ensures even temporal distribution while trimming anomalies in heavily active years.


In [ ]:
# --- Loudness: clip to [-60, 0]
df['loudness'] = df['loudness'].clip(lower=-60, upper=0)

# --- Tempo: clip to [60, 220]
df['tempo'] = df['tempo'].clip(lower=50, upper=220)

# --- Duration: clip to [0, 20] (minutes)
df['duration_mins'] = df['duration_mins'].clip(lower=0, upper=20)

# --- Artist Song Count: clip to [1, 3000]
df['artist_song_count'] = df['artist_song_count'].clip(lower=1, upper=3000)

# --- Genre Song Count: clip to [1, 30000]
df['genre_song_count'] = df['genre_song_count'].clip(lower=0, upper=30000)

# --- Year Song Count: clip to [0, 50000]
df['year_song_count'] = df['year_song_count'].clip(lower=0, upper=50000)


.

.

.

.

.

.

.



**Removing songs from irrelevant genres**

We remove niche, geographically-specific genres and non-musical categories to reduce noise and improve the consistency of the dataset for modelling.

In [ ]:
genresToDrop = ['study','children', 'comedy', 'show-tunes','turkish',
       'german', 'french', 'swedish', 'k-pop',
       'j-rock', 'j-pop', 'cantopop','brazil','latino',
       'disney','mpb','indian','j-dance','anime','mandopop',
       'j-idol','pagode','gospel','afrobeat','ambient',
       'classical','malay','pop-film','iranian','sleep',
       'world-music','spanish','british','salsa','bluegrass',
       'tango','opera','samba','kids','latin','forro','romance',
       'sertanejo','chicago-house','detroit-techno']


for x in range(len(genresToDrop)):
  df = df[df.genre != genresToDrop[x]]


.

.

.

.

.

.

.



###***1.3.5*** - Data Separation

The dataset is separated into three logical components to simplify downstream processing:

- `descriptors_df` stores non-numeric metadata such as song and artist names. These are excluded from modelling but retained for interpretation and output.
- `categorical_df` stores discrete variables such as key, mode, genre and year, which will be one-hot encoded separately.
- The remaining dataset (`df`) contains only continuous features and derived statistics. This later becomes the foundation for feature scaling and model training.

Separating the data at this stage improves modularity and avoids accidental re-use of irrelevant identifiers during model learning.

.

.

Resetting the index after filtering to ensure all of the indexes are cleanly lined up after seperation:

In [ ]:
df.reset_index(drop=True, inplace=True)


.

.

.

.

.

**Descriptive Data**

Separating non-numerical descriptive data into a separate **DataFrame** *descriptors_df*:

In [ ]:
#Listing the columns we want to separate
columnsToDrop=["track_id","artist_name","track_name"]


#creating new empty DataFrame
descriptors_df = pd.DataFrame()


#adding columns to new DataFrame
for column in columnsToDrop:
    descriptors_df[column] = df[column]


#removing the columns from original DataFrame
df = df.drop(labels=columnsToDrop,axis=1)


.

.


In [ ]:
descriptors_df['popularity'] = df['popularity']


.

.

.

.

.

.

.



**Categorical Data**

Separating categorical value data into a separate **DataFrame** *categorical_df*:

In [ ]:
#Listing the columns we want to separate
columnsToDrop=["key","mode","time_signature","year","genre"]


#creating new empty DataFrame
categorical_df = pd.DataFrame()


#adding columns to new DataFrame
for column in columnsToDrop:
    categorical_df[column] = df[column]


#removing the columns from original DataFrame
df = df.drop(labels=columnsToDrop,axis=1)


.

.


In [ ]:
categorical_df['popularity'] = df['popularity']


.

.

.

.

.

.

.



###***1.3.6*** - Additional Datasets

After cleaning and shaping the main dataset, we will split it into multiple specialised versions, each tailored to a different pipeline within the project. These are not random train/test splits, but structured data variants optimised for specific types of analysis or modelling.

- **`pruned_df`** -> *Bias Minimisation Pipeline*  
  This version removes songs whose popularity deviates significantly from the artist's average. It is used in early-stage visualisations and exploratory correlation analysis (Section 2) where pruning improves clarity.

- **`train_df`** -> *Model Training Pipeline*  
  The dataset is one-hot encoded and normalised to allow use in regression and deep learning models. This is the dataset used in all model training tasks (Section 3 onwards).

- **`temporal_df`** -> *Temporal Metadata Pipeline*  
  Here, a randomised decimal is added to each year value to emulate real-world release distribution across time and to reduce temporal data leakage in the dataset. This enables calculation of more realistic running averages for `year_avg_popularity`, `artist_avg_popularity`,`genre_song_count`, etc. which is used in temporal modelling and timeline-based experiments.

Each dataset supports a separate analytical path within the overall architecture of the project, improving both modularity and performance at later stages.




.

.


****

####**A. Introducing rich-featured *pruned_df***

This branch of the pipeline generates a refined subset of the dataset, created by removing entries whose popularity is overly explained by artist-level momentum. Unlike traditional denoising — which focuses on removing noisy or extreme outliers — this pruning procedure actively filters *inliers* that align too closely with the dominant feature `artist_avg_popularity`.

The goal is to eliminate instances where a song's popularity can be largely predicted from the artist's prior success, thereby improving the diversity and learning value of the remaining dataset. A custom filtering function is used to compare each song's actual popularity with its artist's running average, and remove songs that fall within a predefined scale-sensitive symmetry zone. This ensures that models trained later are encouraged to learn from meaningful variation in the features, rather than overfitting to artist prestige.

The result is a more balanced, explanatory subset of the data with fewer redundant or momentum-driven entries. This dataset is used primarily in exploratory correlation analysis in Section *2*, where fairness and feature independence are of interest.

- **`pruned_df`** -> *Bias Minimisation Pipeline*  
  Removes over-aligned inliers based on artist momentum. Used for more balanced correlation inspection and to reduce popularity bias.


In [ ]:
pruned_df = df.copy()

In [ ]:
#"""
ax1 = df.plot.scatter(x='popularity',
                      y='artist_avg_popularity',
                      c='popularity',
                      colormap='viridis')
#"""


.

.


#####**Engineering Pruning Function**

This custom pruning algorithm dynamically removes data points that fall too close to the diagonal correlation between popularity and artist momentum. It identifies a top-right reference point in the feature space — representing the most extreme joint popularity — and constructs a scaled diamond boundary around it. Songs falling within this shape are filtered out as over-aligned, helping to retain only those tracks whose popularity diverges meaningfully from their artist's historical trend.

*Note: This logic is best visualised by referring to the explanatory plots at the top and bottom of this section.*



.

.

.


**Defining *X_Max()* :**

The `X_Max()` function identifies the most extreme composite point in the dataset based on the sum of `popularity` and `artist_avg_popularity`. This point — typically in the top-right of the correlation space — serves as a reference for pruning. It anchors the scaled boundary used to filter out overly momentum-aligned songs.


In [ ]:
#finding the maximum composite popularity between popularity and the average popularity
def X_Max(P,A):
  x_max = 0

  #finds the largest function of  popularity + artist avg. popularity within the dataset
  for x in range(len(P)):
    p = P[x]
    a = A[x]
    p_max = P[x_max]
    a_max = A[x_max]

    if p + a > p_max + a_max:
      x_max = x

  return x_max


.

.

.


**Defining *AbsoluteDiff()* :**

The `AbsoluteDiff()` function returns the absolute difference between a song's actual popularity and its artist's average popularity. It is used as the core measure of deviation when determining whether a track should be pruned based on over-alignment with momentum.


In [ ]:
def AbsoluteDiff(p,a):
  return abs(p-a)


.

.

.


**Defining *Filter()* :**

The `Filter()` function determines whether a song should be kept or pruned based on its deviation from artist momentum. It compares the absolute difference between a track's popularity and its artist's average against dynamically scaled thresholds, anchored to the maximum observed values. If the deviation exceeds either boundary, the track is retained; otherwise, it is filtered out as overly momentum-aligned.

In [ ]:
def Filter(p, a, p_max, a_max, scale):
  if(AbsoluteDiff(p, a) > 0.5 * (abs((0.5 * p_max * scale) - (abs(0.5 * p_max - p)))) or
    AbsoluteDiff(p, a) > 0.5 * (abs((0.5 * p_max * scale) - (abs(0.5 * a_max - a))))):
    return True
  else:
    return False


.

.

.


**Defining *Prune()* :**

The `Prune()` function applies the pruning logic across the entire dataset. It uses `X_Max()` to locate the most extreme joint popularity point and passes each row through the `Filter()` function. Only songs that deviate significantly from their artist's average popularity are retained. The result is a new pruned DataFrame `Q`, containing only entries that carry meaningful, independent popularity signals.

In [ ]:
def Prune(df,P,A,scale):
  p = P.to_numpy()
  a = A.to_numpy()
  Q = []

  x_max = X_Max(p,a)
  p_max = p[x_max]
  a_max = a[x_max]

  for x in range(0,len(p)):
    if (Filter(p[x], a[x], p_max, a_max, scale) == True):
      Q.append(df.iloc[x])

  Q = pd.DataFrame(Q)
  Q.columns = df.columns

  return Q


.

.

.



#####**Pruning the DataFrame**

After applying the `Prune()` function with a calibrated scale value, the resulting `pruned_df` retains only those tracks whose popularity deviates meaningfully from their artist's average. The scatter plot below visualises this relationship, where each point reflects a song's absolute popularity versus its artist's average at release. The pruning effectively removes the tightly clustered, momentum-aligned middle region, revealing two distinct slopes of valid variability.


In [ ]:
pruned_df = Prune(pruned_df,pruned_df['popularity'],pruned_df['artist_avg_popularity'],scale=1.55)

In [ ]:
#"""
ax1 = pruned_df.plot.scatter(x='popularity',
                      y='artist_avg_popularity',
                      c='popularity',
                      colormap='viridis')
#"""

**[END]**

.

.

.

.

.

.

.

.

.

.


****

####**B. Introducing one-hot-encoded normalised *train_df***



This branch of the pipeline is used to prepare the dataset for all supervised learning tasks. It combines both manual one-hot encoding of categorical variables and normalisation of continuous variables, producing a clean, model-ready format referred to as `train_df`.

Manual one-hot encoding is applied to categorical variables including genre, key, mode, and time signature. This is done mostly manually — rather than using automated library functions — to retain full control over the naming of resulting features and to better understand how categorical values are mapped. This step improves both interpretability and downstream UI integration. Each categorical variable is encoded separately and added back to the dataset.

Next, continuous features that fall outside of the 0-1 range are normalised. These variables are scaled based on their known min-max values or within defined bounds to ensure that no feature dominates the model due to scale. The features include loudness, tempo, duration, artist and genre statistics, and temporal context variables.

This version of the dataset forms the core input to all modelling stages in Sections *3* and *4*. It represents the most refined and standardised structure available in the pipeline — fully numerical, bounded, and containing no nulls or unused metadata.

- **`train_df`** -> *Model Training Pipeline*  
  This is the final dataset used for model training. All categorical features are one-hot encoded, continuous values are scaled to [0, 1].

In [ ]:
train_df = df.copy()


.

.

.

.

.

.

.



#####**One-Hot-Encoding**

Manual one-hot encoding is applied to selected categorical features such as genre, key, mode, and time signature. This approach is used instead of a library-based method (e.g. `OneHotEncoder` from `sklearn.preprocessing`) for two main reasons:

- **Naming Control**: Manual encoding allows complete control over the naming of the resulting variables. This is useful for interpretability, presentation, and consistency with later UI integration.

- **Understanding**: Implementing one-hot encoding manually also offers visibility into the structure of the transformation. It ensures a clearer understanding of how categorical information is converted into model-ready numeric format, which is useful for both debugging and learning.

Each categorical variable is encoded independently and merged back into `train_df` once complete.

**For *'key'* :**

In [ ]:
#print(categorical_df["key"].unique())
# 0=C 1=Db 2=D 3=Eb 4=E 5=F 6=Gb 7=G 8=Ab 9=A 10=Bb 11=B

keyOfC = []
keyOfDb = []
keyOfD = []
keyOfEb = []
keyOfE = []
keyOfF = []
keyOfGb = []
keyOfG = []
keyOfAb = []
keyOfA = []
keyOfBb = []
keyOfB = []

#https://www.kaggle.com/code/alankarmahajan/exploring-spotify-dataset

Loop over each row, and mark the correct *key* value with `1.0` :

In [ ]:
n = 0

In [ ]:
for key in categorical_df['key']:

  # Start every column as 0
  keyOfC.append(0)
  keyOfDb.append(0)
  keyOfD.append(0)
  keyOfEb.append(0)
  keyOfE.append(0)
  keyOfF.append(0)
  keyOfGb.append(0)
  keyOfG.append(0)
  keyOfAb.append(0)
  keyOfA.append(0)
  keyOfBb.append(0)
  keyOfB.append(0)

  # Switch the correct key column to 1 if the key is present for that row
  if key == 0:
    keyOfC[n] = 1.0
  elif key == 1:
    keyOfDb[n] = 1.0
  elif key == 2:
    keyOfD[n] = 1.0
  elif key == 3:
    keyOfEb[n] = 1.0
  elif key == 4:
    keyOfE[n] = 1.0
  elif key == 5:
    keyOfF[n] = 1.0
  elif key == 6:
    keyOfGb[n] = 1.0
  elif key == 7:
    keyOfG[n] = 1.0
  elif key == 8:
    keyOfAb[n] = 1.0
  elif key == 9:
    keyOfA[n] = 1.0
  elif key == 10:
    keyOfBb[n] = 1.0
  elif key == 11:
    keyOfB[n] = 1.0
  else:
    print("ERROR: Reason='erroneous value'")
    break

  # Accumulator increases
  n += 1

Adding the finished arrays to the dataset:

In [ ]:
train_df['keyOfC'] = keyOfC
train_df['keyOfDb'] = keyOfDb
train_df['keyOfD'] = keyOfD
train_df['keyOfEb'] = keyOfEb
train_df['keyOfE'] = keyOfE
train_df['keyOfF'] = keyOfF
train_df['keyOfGb'] = keyOfGb
train_df['keyOfG'] = keyOfG
train_df['keyOfAb'] = keyOfAb
train_df['keyOfA'] = keyOfA
train_df['keyOfBb'] = keyOfBb
train_df['keyOfB'] = keyOfB


.

.

.

.

.

.

.



**For *'mode'* :**

In [ ]:
#print(categorical_df["mode"].unique())
# 0=Minor 1=Major

Minor = []
Major = []

#https://www.kaggle.com/code/alankarmahajan/exploring-spotify-dataset

*Loop* over each row, and mark the correct *mode* value with `1.0` :

In [ ]:
n = 0

In [ ]:
#we have our options for mode - 0(minor) or major(1) - so we split mode into the following individual arrays
#that will contain a 1(true) or a 0(false) in them to note if that row is the correct mode or not.

for x in categorical_df['mode']:

  if x == 0:
    Minor.append(1.0)             #if the mode is Minor, then it will be set to a 1 for this row.
    Major.append(0)

  elif x == 1:
    Minor.append(0)
    Major.append(1.0)             #if the mode is Major, then it will be set to a 1 for this row.

  else:
    print("ERROR: Reason='erroneous value'")
    break

    #accumulator increases
    n += 1

Adding the finished arrays to the dataset:

In [ ]:
train_df['minor'] = Minor
train_df['major'] = Major


.

.

.

.

.

.

.



**For *'time_signature'* :**

In [ ]:
#print(categorical_df["time_signature"].unique())
# 3 = 3/4,    4 = 4/4,    5 = 5/4

_34time = []
_44time = []
_54time = []

#https://community.spotify.com/t5/Spotify-for-Developers/Section-time-signature/td-p/5180561

Loop over each row, and mark the correct *time_signature* value with `1.0` :

In [ ]:
n = 0

In [ ]:
#this for loop splits each letter into one column.
for x in categorical_df['time_signature']:

  if x == 3:
    _34time.append(1.0)     #if the time signature is 3/4 it will be set to a 1 for this row.
    _44time.append(0)
    _54time.append(0)

  elif x == 4:
    _34time.append(0)
    _44time.append(1.0)     #if the time signature is 4/4, it will be set to a 1 for this row.
    _54time.append(0)

  elif x == 5:
    _34time.append(0)
    _44time.append(0)
    _54time.append(1.0)     #if the time signature is 5/4, it will be set to a 1 for this row.

  else:
    print("ERROR: Reason='erroneous value'")
    break

  #accumulator increases
  n += 1

In [ ]:
#adding the finished arrays to the dataset
train_df['time_signature_34'] = _34time
train_df['time_signature_44'] = _44time
train_df['time_signature_54'] = _54time


.

.

.

.

.

.

.



**For *'genre'* and *'year'* :**

For the `genre` and `year` columns, the existing category names are already clear and interpretable, so there's no need to manually encode them. Instead, we use scikit-learn's built-in `OneHotEncoder`, which efficiently handles the rest of the one-hot transforming.

In [ ]:
genre_year = ['genre','year']
encoder = OneHotEncoder(sparse_output=False)

In [ ]:
one_hot_genre_year = pd.DataFrame(
    (encoder.fit_transform(categorical_df[genre_year])),
    columns = encoder.get_feature_names_out(genre_year))

In [ ]:
train_df = pd.concat([train_df, one_hot_genre_year],axis=1)
one_hot_genre_year['popularity'] = df['popularity']


.

.

.

.

.

.

.


#####**Normalisation**

In this subsection, we scale a subset of continuous features so that they fall within the 0-1 range. This is an essential step before modelling, particularly for algorithms like neural networks, which are sensitive to the scale of their input features. Without normalisation, features with larger numeric ranges (like song count or loudness) could disproportionately affect the model and reduce training stability.

The following variables are selected for normalisation based on their observed min-max values or known practical bounds. Features are clipped already from *1.3.4 Data Filtering*, giving us bounds to normalise within to ensure that the scaled values remain meaningful.

This ensures that all features contribute proportionally to model training and that the optimisation process remains stable and efficient.

In [ ]:
#                            min           max
# acousticness               0.000000      0.996000
# danceability               0.000000      0.993000
# energy                     0.000000      1.000000
# instrumentalness           0.000000      1.000000
# liveness                   0.000000      0.700000
# loudness---------------- -47.283000      0.000000  NORMALISE
# speechiness                0.000000      0.967000
# tempo-------------------  50.000000    220.000000  NORMALISE
# valence                    0.000000      1.000000
# artist_avg_popularity---   0.000000     85.000000  NORMALISE
# artist_song_count-------   1.000000   1786.000000  NORMALISE
# genre_avg_popularity----   0.001277     55.772119  NORMALISE
# genre_song_count-------- 525.000000  20964.000000  NORMALISE
# year_avg_popularity-----  11.955374     33.103448  NORMALISE
# year_song_count------- 25736.000000  37295.000000  NORMALISE
# duration_mins-----------   0.034550     20.000000  NORMALISE
# popularity--------------   0.000000    100.000000  NORMALISE

The following features require normalisation because they do not naturally fall within a 0-1 range. Each is scaled to [0, 1] based on its minimum and maximum observed values, or within known bounds. This ensures all features contribute proportionally during model training, avoiding dominance from variables with large raw scales.

- **Loudness**: min = -47.28, max = 0  
  Measured in decibels. Normalised after clipping from [−60, 0] to [0, 1] to reflect realistic audio volume bounds.

- **Tempo**: min = 50, max = 220  
  Measured in beats per minute. Normalised after clipping from [50, 220] to [0, 1] to cover the practical tempo range used in most recorded music.

- **Duration (minutes)**: min = 0.03, max = 20  
  Converted from milliseconds. Normalised after clipping from [0, 20] to [0, 1] to exclude extreme outlier durations (e.g. live sets or experimental pieces).

- **Artist Average Popularity**: min = 0, max = 85  
  Represents the average popularity score across each artist's discography. Normalised from [0, 100] to [0, 1] to keep in proportion with other model features.

- **Artist Song Count**: min = 1, max = 1786  
  Indicates the number of songs attributed to an artist. Normalised after clipping from [0, 3000] to [0, 1] to prevent prolific artists from skewing the distribution.

- **Genre Average Popularity**: min = 0.001, max = 55.77  
  Reflects the average popularity of all songs within a genre. Normalised after clipping from [0, 100] to [0, 1] to balance genre-level influence.

- **Genre Song Count**: min = 525, max = 20,964  
  Number of songs attributed to a genre. Normalised after clipping from [0, 30,000] to [0, 1] to avoid dominance by the most common genres.

- **Year Average Popularity**: min = 11.96, max = 33.10  
  Represents the average popularity of songs released in a given year. Normalised after clipping from [0, 100] to [0, 1] to align with other inputs.

- **Year Song Count**: min = 25,736, max = 37,295  
  The number of songs released per year. Normalised after clipping from [0, 50,000] to [0, 1] to mitigate overrepresentation from more recent years.

- **Popularity**: min = 0, max = 100  
  The target variable used for prediction. Normalised from [0, 100] to [0, 1] for consistency with the rest of the feature set.



.

.

.


**Manual Scaling** unnormalised features to [0,1]

In [ ]:
# Manual scaling for selected features

# --- Loudness: [-60, 0] scale to [0, 1]
train_df['loudness'] = (train_df['loudness'] + 60) / 60

# --- Tempo: [60, 220] scale to [0,1]
train_df['tempo'] = (train_df['tempo'] - 50) / (220 - 50)

# --- Duration: [0, 20] scale to [0,1]
train_df['duration_mins'] = (train_df['duration_mins']) / 20

# --- Popularity: scale from [0, 100] to [0, 1]
train_df['popularity'] = train_df['popularity'] / 100

# --- Artist Average Popularity: scale from [0, 100] to [0, 1]
train_df['artist_avg_popularity'] = train_df['artist_avg_popularity'] / 100

# --- Genre Average Popularity: scale from [0, 100] to [0, 1]
train_df['genre_avg_popularity'] = train_df['genre_avg_popularity'] / 100

# --- Year Average Popularity: scale from [0, 100] to [0, 1]
train_df['year_avg_popularity'] = train_df['year_avg_popularity'] / 100

# --- Artist Song Count: [0, 3000] scale to [0, 1]
train_df['artist_song_count'] = (train_df['artist_song_count']) / 3000

# --- Genre Song Count: [0, 30000] scale to [0, 1]
train_df['genre_song_count'] = (train_df['genre_song_count']) / 30000

# --- Year Song Count: [0, 50000] scale to [0, 1]
train_df['year_song_count'] = (train_df['year_song_count']) / 50000



.

.


In [ ]:
display(train_df)

**[END]**

.

.

.

.

.

.

.

.

.

.


****

####**C. Introducing time emulated *temporal_df***

This part of the pipeline prepares `temporal_df`, a time-aware version of the dataset used for exploring trends and building features that respect chronological order.



A `year_decimal` column is added to simulate realistic release distribution across the calendar year, allowing smoother time-based analysis. We have to do it this way because the dataset being used doesn't include a release date feature - the finest temporal data included is by year - so we essentially just impute our release dates. A custom `calculate_running_average()` function is then used to compute expanding averages and song counts globally, and within artist, genre, and year groups. These values exclude the current song using a `.shift(1)` to ensure that the data is calculated from immediately before the release, thus fully respecting time.

The resulting dataset supports temporally safe exploration and helps avoid leakage in downstream analysis. It's used mainly in Sections ***2*** and ***4*** for visualisations and interpretive correlation work.

- **`temporal_df`** -> *Temporal Metadata Pipeline*  
  Contains `year_decimal`, plus historical averages and counts for artist, genre, and year.



.

.


In [ ]:
temporal_df = df.copy()

In [ ]:
temporal_df['year'] = categorical_df['year']
temporal_df['artist_name'] = descriptors_df['artist_name']
temporal_df['genre'] = categorical_df['genre']

In [ ]:
#"""
# Popularity against Year
ax1 = temporal_df.plot.scatter('year', 'popularity', c='year', cmap='viridis')
ax1.set_title('Popularity vs Year')
ax1.set_xlabel('Year')
ax1.set_ylabel('Popularity')
plt.tight_layout()
plt.show()
#"""


.

.

.

.

.

#####**Calculating Time Emulated Popularity (Year-Based Running Averages):**

To simulate a realistic time-based context for each song, a fractional component is added to the release year (`year_decimal`) to emulate intra-year variability. This enabled the calculation of temporally-aware running averages and song counts using a custom `calculate_running_average()` function. These statistics are computed globally, and within groups (artist, genre, year), using `.expanding().mean().shift(1)` to ensure each value only reflects prior data — preventing temporal leakage.


**Adding a realistic decimal for intra-year time spreading:**

In [ ]:
year_decimal = []
for year in temporal_df['year']:
    randomDecimal = round((1/365) * random.randint(0, 364), 3)
    year_decimal.append(year + randomDecimal)

temporal_df['year_decimal'] = year_decimal

In [ ]:
display(temporal_df[['artist_name', 'popularity', 'year', 'year_decimal']])


.

.

.

.

.


**Defining Function *calculate_running_average()*** for calculating the running-average and running total (by group, or by whole dataset):

In [ ]:
def calculate_running_average(df, group_name=None, display_name="running"):

  # (IF GROUP_NAME SELECTED): Sort by group and time
  if group_name != None:
    df = df.sort_values(by=[group_name, 'year_decimal', 'running_song_count']).copy()

    # Compute expanding average and count
    grouped = df.groupby(group_name)['popularity']

    avg_popularity = grouped.expanding().mean().shift(1).reset_index(level=0, drop=True)
    song_count = grouped.expanding().count().shift(1).reset_index(level=0, drop=True)

    # Assign to DataFrame
    df[f'{display_name}_avg_popularity'] = avg_popularity
    df[f'{display_name}_song_count'] = song_count

    # Explicitly force 0 for first song of each artist
    first_song_index = df.groupby(group_name).head(1).index

    for index in first_song_index:
        df.loc[index, f'{display_name}_avg_popularity'] = 0
        df.loc[index, f'{display_name}_song_count'] = 0
    return df


  # (NO GROUP_NAME SELECTED): In this instance, we just compute the running values on the whole dataset
  else:
    df = df.sort_values(by='year_decimal').copy()

    # Calculate expanding average and count across the full dataset
    popularity = df['popularity']

    avg_popularity = popularity.expanding().mean().shift(1)
    song_count = popularity.expanding().count().shift(1)

    # Assign to DataFrame
    df[f'{display_name}_avg_popularity'] = avg_popularity
    df[f'{display_name}_song_count'] = song_count

    # First row (index 0 after sort) gets forced 0
    first_index = df.index[0]
    df.loc[first_index, f'{display_name}_avg_popularity'] = 0
    df.loc[first_index, f'{display_name}_song_count'] = 0
    return df



.

.

.

.

.


#####**Adding Time-Respective Running Metadata to *temporal_df***

To enrich `temporal_df` with realistic historical context and minimise temporal data leakage, group-based running averages and song counts are calculated across key metadata dimensions: artist, genre, and year. Each uses an expanding mean excluding the current row, ensuring temporal causality. These variables simulate what would have been historically knowable at each point in time, and are used for interpretive analysis and correlation visualisation in the project.


.

.


Calculating Global Running Metadata:

In [ ]:
temporal_df = calculate_running_average(temporal_df)

In [ ]:
display(temporal_df[['year', 'artist_name', 'year_decimal', 'popularity', 'running_avg_popularity', 'running_song_count']].sort_index().sort_values(by=['year_decimal', 'running_song_count']))


.

.

.


Calculating Running Artist Metadata:

In [ ]:
temporal_df = calculate_running_average(temporal_df, group_name='artist_name', display_name='artist')

In [ ]:
#specific artist's progression
new_df = temporal_df[temporal_df['artist_name'] == 'Taylor Swift']
display(new_df[['artist_name', 'genre', 'year_decimal', 'popularity', 'artist_avg_popularity', 'artist_song_count', 'running_song_count']].sort_index().sort_values(by='artist_song_count'))


.

.

.


Calculating Running Genre Metadata:

In [ ]:
temporal_df = calculate_running_average(temporal_df, group_name='genre', display_name='genre')

In [ ]:
#specific genre's progression
display(temporal_df[temporal_df['genre'] == 'rock'][['genre', 'artist_name', 'year_decimal', 'popularity', 'genre_avg_popularity', 'genre_song_count']].sort_index().sort_values(by='genre_song_count'))


.

.

.


Calculating Running Year Metadata:

In [ ]:
temporal_df = calculate_running_average(temporal_df, group_name='year', display_name='year')

In [ ]:
#a specific years' progression
new_df = temporal_df[temporal_df['year'] > 2020]
display(new_df[['year', 'artist_name', 'year_decimal', 'popularity', 'year_avg_popularity', 'year_song_count']].sort_index().sort_values(by=['year', 'year_song_count']))


.

.


`artist_name` and `genre` columns were just for generating the metadata - now we can remove them
so that we only have continuous data in temporal_df:

In [ ]:
temporal_df = temporal_df.drop(columns=['artist_name','genre'])


.

.

.

.

.



**Displaying *temporal_df* :**

In [ ]:
display(temporal_df[['popularity', 'year_decimal', 'artist_avg_popularity', 'artist_song_count',
                     'genre_avg_popularity', 'genre_song_count', 'year_avg_popularity',
                     'year_song_count', 'running_avg_popularity', 'running_song_count']
                    ])

In [ ]:
#"""
# Popularity against Year_Decimal
ax1 = temporal_df.plot.scatter('year_decimal', 'popularity', c='year', cmap='viridis')
ax1.set_title('Popularity vs Year_Decimal')
ax1.set_xlabel('Year_Decimal')
ax1.set_ylabel('Popularity')
plt.tight_layout()
plt.show()
#"""

**[END]**

.

.

.

.

.

.

.

.

.

.


#**Section 2** - Exploratory Analysis

##**2.1** Distribution and Basic Statistics

In this section, we examine the distributions and statistical properties of the dataset's features to understand their underlying characteristics. This analysis provides insights into the central tendencies, variability, and shape of the data distributions, which are crucial for subsequent modelling, research and interpretation.



.

.

.


###***2.1.1*** - Histograms of All Features (Distributed by Genre)

These histograms provide a first look at the distribution of each numerical feature in the dataset. Most variables (e.g. `danceability`, `energy`, `valence`) are already bounded between 0 and 1 and display reasonable spread, although some lean towards skewed distributions (e.g. `acousticness`, `instrumentalness`, `liveness`).

Others, like `loudness`, `tempo`, and various *_count* or *_popularity* fields, span wider numerical ranges. These distributions justify the need for normalisation, especially in features where outliers or long tails are present.

From a modelling perspective, understanding the spread helps anticipate which features may require transformation or carry more weight during prediction. It also highlights where features might dominate due to scale, or where certain values cluster unnaturally (e.g. high concentration at zero for `instrumentalness`).



.

.


This code iterates through all numerical columns in the dataset, plotting their distributions to identify any skewness, outliers, or unusual patterns.

In [ ]:
# Count how many times each genre appears
data = df.copy()
f64_col = data.columns.tolist()
data['genre'] = categorical_df['genre']
tmp = data['genre'].value_counts()
tmp = pd.DataFrame(tmp)
tmp.columns = ['count']

# Add percentage column
tmp['frequency'] = tmp['count'] / tmp['count'].sum()
tmp['frequency'] = tmp['frequency'].apply(lambda x: f'{x*100:.2f}%')

# Calculate average popularity per genre
genre_popularity = data.groupby('genre')['popularity'].mean()
genre_popularity = pd.DataFrame(genre_popularity)

# Merge into tmp
tmp = tmp.merge(genre_popularity, left_index=True, right_index=True)

# Get top 15 genre names
all_genre = list(tmp.sort_values(by='popularity', ascending=False).index)
top15_genre = list(tmp.sort_values(by='popularity', ascending=False).head(15).index)

#f64_col = data.select_dtypes(include=['float64', 'int64']).columns.tolist()

# This code at it's core is not mine - the source can be found below.
#https://www.kaggle.com/code/hmdmin357/spotify-classification-analysis/notebook

In [ ]:
def make_hist(pal, genre):
    data15=data[data['genre'].isin(genre)]
    c_order={x:i+1 for i,x in enumerate(genre)}
    palette=sns.color_palette('Set3',n_colors=len(genre))
    palette2=sns.color_palette('hsv',n_colors=len(genre))[::-1]
    palette3=[]
    for i in range(len(genre)):
        palette3.append((palette[i][0]*0.8+palette2[i][0]*0.2,palette[i][1]*0.8+palette2[i][1]*0.2,palette[i][2]*0.8+palette2[i][2]*0.2))
    data15=data15.assign(c_order=data15['genre'].map(c_order)).sort_values('c_order').drop('c_order',axis=1)
    plt.figure(figsize=(40,30))
    sns.histplot(data=data15,x=pal,hue='genre',multiple='stack',binwidth=0.02*(data15[pal].max()-data15[pal].min()),palette=palette3)
    plt.tight_layout()
    plt.title(pal.title() +' Histogram in the Top-15 Genres')
    plt.show()


# This code is not mine - the source can be found below.
#https://www.kaggle.com/code/hmdmin357/spotify-classification-analysis/notebook


.

.

.


**Feature Distribution by Genre:**

Below we generate histograms for all continuous audio features, segmented by the top 15 most popular genres. This allows us to visually compare how these features are distributed across genre types and observe stylised distinctions between categories.


In [ ]:
for pal in f64_col:
    if pal!='genre':
        make_hist(pal,all_genre)

# This code is not mine - the source can be found below.
#https://www.kaggle.com/code/hmdmin357/spotify-classification-analysis/notebook


.

.


In [ ]:
for pal in f64_col:
    if pal!='genre':
        make_hist(pal,top15_genre)

# This code is not mine - the source can be found below.
#https://www.kaggle.com/code/hmdmin357/spotify-classification-analysis/notebook


.

.

.

.

.


###***2.1.2*** - Summary Statistics

Following the visual inspection of feature distributions, this summary statistics table provides a more quantitative view of central tendencies and spread. Features like `danceability`, `energy`, and `valence` continue to show compact, bounded ranges, which is expected given their design as normalised values.

In contrast, variables such as `artist_song_count`, `year_song_count`, `loudness`, and `duration_mins` span much wider scales. Their interquartile ranges and high maximum values reinforce the earlier observation that scale disparities could distort model behaviour without proper normalisation.

We also calculate skewness and kurtosis to quantify distribution shape. Features like `instrumentalness` and `loudness` exhibit significant skew or heavy tails, suggesting non-normal distributions with outlier influence.

In [ ]:
# Display summary statistics for numerical features
summary_stats = df.describe().T
summary_stats['skewness'] = df.skew()
summary_stats['kurtosis'] = df.kurtosis()
summary_stats = summary_stats[['mean', 'std', 'min', '25%', '50%', '75%', 'max', 'skewness', 'kurtosis']]
summary_stats


.

.

.


In [ ]:
df.describe()


.

.

.



.

.

.

.

.


###***2.1.3*** - Skewness and Kurtosis Analysis

To extend the statistical view beyond central tendency and spread, skewness and kurtosis were calculated for each continuous feature. Skewness captures asymmetry — whether values lean heavily to one side of the mean — while kurtosis reflects how extreme or concentrated the tails are compared to a normal distribution.

Several features show significant skew (>|1|), including `loudness`, `speechiness`, `acousticness`, `liveness`, `artist_song_count`, and `duration_mins`. These are not symmetrically distributed, with many skewed to the right — indicating a large cluster of lower values and a long tail of higher ones. The same features (especially `artist_song_count`) also show high kurtosis (>|3|), suggesting sharp peaks or heavy tails with influential outliers.

These patterns reinforce earlier findings from the histograms and summary stats, and support the later decision to clip extreme values and apply normalisation — particularly for features likely to distort model training if left unscaled.


In [ ]:
# Identify features with high skewness or kurtosis
high_skew = summary_stats[summary_stats['skewness'].abs() > 1]
high_kurtosis = summary_stats[summary_stats['kurtosis'].abs() > 3]

print("Features with high skewness:")
display(high_skew)

print("\n\nFeatures with high kurtosis:")
display(high_kurtosis)


.

.

.

.

.


###***2.1.4*** Distribution by Average Popularity

Here we break down popularity by genre, year, and artist to get a sense of broader trends and patterns in the data. This helps identify whether certain genres or time periods consistently perform better, and whether some artists or genres, etc. dominate the dataset.



.

.

.


**By average genre popularity *genre_avg_popularity* :**

This plot shows the average popularity of songs across all genres in the dataset. Genres like pop, hip-hop, and dance dominate in terms of average popularity, which likely reflects broader listening trends and platform bias. Less mainstream or instrumental-heavy genres like trip-hop, dubstep, and black-metal sit at the lower end.


In [ ]:
genre_columns = [col for col in train_df.drop(columns='genre_avg_popularity',axis=1).columns if col.startswith('genre_')]


genre_popularity = train_df.groupby(
    train_df[genre_columns].idxmax(axis=1).str.replace('genre_', '')
)['popularity'].mean().reset_index()

# Rename the index column to 'genre'
genre_popularity.rename(columns={'index': 'genre'}, inplace=True)

# Sort by popularity
genre_popularity = genre_popularity.sort_values(by='popularity', ascending=False)

# Create the bar plot
plt.figure(figsize=(10, 6))
sns.barplot(x='genre', y='popularity', data=genre_popularity, hue='genre', palette='viridis')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for readability
plt.title('Average Popularity by Genre')
plt.tight_layout()
plt.show()


.

.

.


**By average yearly popularity *year_avg_popularity* :**

This bar chart shows a clear upward trend in average popularity over time, with more recent years achieving higher values. This reflects both recency bias in Spotify's popularity metric, and potentially the increasing influence of recommendation systems favouring newer releases.


In [ ]:
# Get year columns
year_columns = [col for col in train_df.drop(
    columns=['popularity','year_song_count'], axis=1
).columns if col.startswith('year_')]

# Create DataFrame for year popularity
year_popularity = train_df.groupby(
    train_df[year_columns].idxmax(axis=1).str.replace('year_', '')
)['popularity'].mean().reset_index()

# Rename for clarity
year_popularity.columns = ['year', 'popularity']

# Convert year to int for sorting
year_popularity['year'] = year_popularity['year'].astype(int)

# Sort by popularity
year_popularity = year_popularity.sort_values(by='popularity', ascending=False)

# Plot
plt.figure(figsize=(12, 6))
ax = sns.barplot(x='year', y='popularity', data=year_popularity, hue='year', palette='viridis', legend=False)
plt.xticks(rotation=45, ha='center')



plt.title('Average Popularity by Year (2000-2022)')
plt.tight_layout()
plt.show()



.

.

.


**Average Artist Popularity *artist_avg_popularity* by Artist *artist_name* :**

These are the top 30 most popular artists in the dataset based on average popularity across all of their songs. The list is dominated by newer, high-performing artists with viral hits or consistent playlist presence. This result isn't weighted by number of songs - it's purely based on artist-level average popularity.


In [ ]:
# Group by artist name and calculate average popularity with a limit of atleast 10 songs

# Add artist names into df
artist_popularity = df.copy()
artist_popularity['artist_name'] = descriptors_df['artist_name']
artist_popularity = artist_popularity[artist_popularity['artist_song_count'] > 10]

# Drop duplicates so each artist appears once with their true artist_avg_popularity
artist_popularity = artist_popularity.drop_duplicates(subset='artist_name')

# Sort by artist_avg_popularity
artist_popularity = artist_popularity.sort_values(by='artist_avg_popularity', ascending=False)

# Get top 30 artists
top30_artists = artist_popularity.head(30)

In [ ]:
# Create the bar plot for top 30 Artists
plt.figure(figsize=(10, 6))
sns.barplot(x='artist_name', y='artist_avg_popularity', data=top30_artists, hue='artist_name', palette='viridis')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for readability
plt.title('Top 30 Artists by Artist Average Popularity (2000-2023)')
plt.tight_layout()
plt.show()


.

.

.


This adjusted list includes only the top 30 artists with larger catalogues (> 100 songs), helping to filter out viral or short-term outliers. It highlights artists who have sustained popularity over time, such as Drake, Taylor Swift, Ariana Grande, and Kendrick Lamar.


In [ ]:
# Add artist names into df
matured_artist_popularity = df.copy()
matured_artist_popularity['artist_name'] = descriptors_df['artist_name']
matured_artist_popularity = matured_artist_popularity[matured_artist_popularity['artist_song_count'] > 100]

# Drop duplicates so each artist appears once with their true artist_avg_popularity
matured_artist_popularity = matured_artist_popularity.drop_duplicates(subset='artist_name')

# Sort by artist_avg_popularity
matured_artist_popularity = matured_artist_popularity.sort_values(by='artist_avg_popularity', ascending=False)

# Get top 50 artists
top30_matured_artists = matured_artist_popularity.head(30)

In [ ]:
# Create the bar plot for top 30 Artists (Matured)
plt.figure(figsize=(10, 6))
sns.barplot(x='artist_name', y='artist_avg_popularity', data=top30_matured_artists, hue='artist_name', palette='viridis')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for readability
plt.title('Top 30 Artists (Matured) by Artist Average Popularity (2000-2023)')
plt.tight_layout()
plt.show()


.

.

.


This is a broad sample of the full artist distribution, ordered by artist average popularity. The long-tail distribution is clearly visible, where a few artists sit at the top, but most have relatively low average popularity.


In [ ]:
all_artist_popularity = df.sample(n=1000).copy()
all_artist_popularity['artist_name'] = descriptors_df['artist_name'].loc[all_artist_popularity.index]
all_artist_popularity = all_artist_popularity.drop_duplicates(subset='artist_name')
all_artist_popularity = all_artist_popularity.sort_values(by='artist_avg_popularity')

In [ ]:
# Create the bar plot for artist distribution
plt.figure(figsize=(10, 6))
sns.barplot(x='artist_name', y='artist_avg_popularity', data=all_artist_popularity, legend=False)
plt.title('All Artists (Represented by Sample of ~1000) by Artist Average Popularity (2000-2023)')
plt.xticks([], [])
plt.tight_layout()
plt.show()

**[END]**

.

.

.

.

.

.

.

.

.

.


##**2.2** Correlation and Feature Pairs

In this section, we examine how individual features relate to each other and, more importantly, to the target variable - `popularity`. Understanding correlation is vital for identifying redundant features, multicollinearity, and variables that may strongly or weakly influence model predictions.

This process heavily supports fundamental understanding of the data's inner relationships.


.

.

.


###***2.2.1*** - Correlation Matrix (Continuous)

This correlation matrix reveals the linear relationships between all continuous features in the dataset. As expected, `artist_avg_popularity` shows the strongest positive correlation with `popularity` (r = 0.84), confirming its role as a dominant predictor. Similarly, `genre_avg_popularity` and `year_avg_popularity` are also positively correlated, supporting the idea that broader trends (artist, genre, time) significantly influence a song's success.

Conversely, features like `instrumentalness`, `duration_mins`, and `artist_song_count` have slight to moderate negative correlations with `popularity`, suggesting that long tracks, instrumentals, or overly prolific artists may be less associated with mainstream appeal.

These findings guide feature prioritisation for modelling and help identify variables where multicollinearity may be present (e.g. `energy` and `loudness`, r = 0.79).


In [ ]:
# Create a correlation matrix for continuous features
corr_matrix = df.corr(numeric_only=True)

# Plot the correlation matrix
plt.figure(figsize=(14, 10))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True)
plt.title("Correlation Matrix (Continuous Features)")
plt.tight_layout()
plt.show()


.

.

.


This bar chart shows the top 5 strongest positive correlations with popularity from the main correlation matrix (`df`). As we know already, `artist_avg_popularity` and `genre_avg_popularity` are the most dominant - these are heavily biased metadata features that reflect external popularity factors. `year_avg_popularity` and `year_song_count` show a temporal influence, while `danceability` is the strongest musical feature here, though its correlation is much weaker overall.


In [ ]:
pop_corr_df = corr_matrix['popularity'].drop('popularity')
pop_corr_df = pop_corr_df.sort_values()


# Plot top 5 positive
plt.figure(figsize=(10, 5))
pop_corr_df.tail(5).plot(kind='barh', color='green')
plt.title('Top 5 Positive Correlations (Continuous)')
plt.xlabel('Correlation')
plt.tight_layout()
plt.show()


# Plot top 5 negative
plt.figure(figsize=(10, 5))
pop_corr_df.head(5).plot(kind='barh', color='red')
plt.title('Top 5 Negative Correlations (Continuous)')
plt.xlabel('Correlation')
plt.tight_layout()
plt.show()


.

.

.

.

.


###***2.2.2*** - Correlation Matrix (Categorical)

This extended correlation matrix reveals the linear relationships between continuous features and one-hot encoded categorical indicators such as genre and year. As expected, `artist_avg_popularity` again shows the strongest positive correlation with `popularity` (r = 0.84), followed by `genre_avg_popularity` and `year_avg_popularity`, reinforcing the influence of broader metadata signals on track success.

Notably, strong correlations also emerge between `genre_hip-hop`, `genre_dance`, and recent years like `year_2022`, suggesting modern genres are more aligned with popularity. This supports the idea of recency bias and genre-driven platform trends shaping outcomes.

Conversely, categorical features such as `genre_black-metal` or earlier year indicators show weaker or negative correlations with `popularity`, implying a lower likelihood of mainstream visibility. These insights help validate the inclusion of one-hot categorical inputs and inform which features may offer the most predictive value without introducing redundancy.


In [ ]:
# Create a correlation matrix for continuous and categorical features
one_hot_corr_matrix = train_df.corr(numeric_only=True)

In [ ]:
# Get correlations with popularity only
pop_corr = one_hot_corr_matrix['popularity'] # remove self-correlation

# Take absolute values to find strongest correlations regardless of direction
pop_corr = pop_corr.abs()

# Sort and get top 30
top_features = pop_corr.sort_values(ascending=False).head(30)


In [ ]:
selected = one_hot_corr_matrix.loc[top_features.index, top_features.index]

# Plot the correlation matrix
plt.figure(figsize=(24, 24))
sns.heatmap(selected, annot=True, fmt=".2f", cmap='coolwarm', square=True)
plt.title("Sample of Correlation Matrix (inc. Categorical Features)") # includes a sample of the top 30 relevant features,
plt.show()                                                            # as there is too many features to display fully effectively



.

.

.


This plot highlights the top 10 categorical one-hot features most positively correlated with `popularity`. Genre tags like `genre_hip-hop`, `genre_dance`, and `genre_pop` show the strongest relationships, aligning with dominant commercial genres on streaming platforms. Temporal indicators such as `year_2022` and `year_2021` also appear, reinforcing the influence of recency. These correlations help justify the inclusion of selected genre and year indicators as input features during modelling.


In [ ]:
filtered_cols = []
for col in one_hot_corr_matrix.columns:
    if col != 'popularity':
        if col not in df.columns:
            filtered_cols.append(col)

pop_corr_one_hot = one_hot_corr_matrix.loc[filtered_cols, 'popularity']
pop_corr_one_hot = pop_corr_one_hot.sort_values()


# Plot top 10 positive
plt.figure(figsize=(10, 5))
pop_corr_one_hot.tail(10).plot(kind='barh', color='green')
plt.title('Top 10 Positive Correlations (Categorical)')
plt.xlabel('Correlation')
plt.tight_layout()
plt.show()


# Plot top 10 negative
plt.figure(figsize=(10, 5))
pop_corr_one_hot.head(10).plot(kind='barh', color='red')
plt.title('Top 10 Negative Correlations')
plt.xlabel('Correlation')
plt.tight_layout()
plt.show()


.

.

.

.

.


###***2.2.3*** Correlation Matrix (Temporal)

This extended correlation matrix visualises the linear relationships between all continuous features within the temporally-enhanced dataset `temporal_df`. Compared to the standard version derived from `df`, this view incorporates time-respective metadata — including `artist_avg_popularity`, `genre_avg_popularity`, `year_avg_popularity`, and their respective song counts — calculated using running averages at each point in time.

As expected, `artist_avg_popularity` shows a strong positive correlation with `popularity` (r = 0.75), though notably lower than in the static correlation matrix (r = 0.85). This suggests that time-awareness and data leakage minimisation has weakened the direct dependency on artist momentum, which will in turn also enhance the independence of feature influence.

Similarly, `year_avg_popularity` exhibits a more moderate correlation with `popularity` (r = 0.39) compared to their static counterparts, wheras `genre_avg_popualrity` has transformed to become the most influencial correlation of all — a surprisingly durastic change.

Temporal variables like `year_decimal` and `running_song_count` show weaker yet steady correlations with `popularity`, further consolidating the implicit influence of recency and global growth trends in the dataset.

Compared to the original matrix, the temporal version reveals a more balanced and temporally faithful view of feature relationships — essentially avoiding leakage in modelling, and for understanding which features genuinely carries explanatory power in a realistic, time-aware context.

In [ ]:
temporal_corr_matrix = temporal_df.corr(numeric_only=True)
plt.figure(figsize=(19, 13))
sns.heatmap(temporal_corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True)
plt.title("Correlation Matrix (Temporal Continuous Features)")
plt.tight_layout()
plt.show()


.

.

.


This bar chart confirms that the strongest temporal correlations with `popularity` align with metadata-driven variables. `artist_avg_popularity` and `genre_avg_popularity` remain the top predictors, while `year_avg_popularity`, `running_avg_popularity`, and `running_song_count` also contribute meaningfully. These results support the potential for inclusion of time-aware engineered features from `temporal_df` in modelling.


In [ ]:
pop_corr_temporal = temporal_corr_matrix['popularity'].drop('popularity')
pop_corr_temporal = pop_corr_temporal.sort_values()


# Plot top 5 positive
plt.figure(figsize=(10, 5))
pop_corr_temporal.tail(5).plot(kind='barh', color='green')
plt.title('Top 5 Positive Correlations (temporal_df - Temporal)')
plt.xlabel('Correlation')
plt.tight_layout()
plt.show()


# Plot top 5 negative
plt.figure(figsize=(10, 5))
pop_corr_temporal.head(5).plot(kind='barh', color='red')
plt.title('Top 5 Negative Correlations (temporal_df - Temporal)')
plt.xlabel('Correlation')
plt.tight_layout()
plt.show()


.

.

.

.

.


###***2.2.4*** - Exploring Correlated Feature Pairs

This section presents the key pairwise correlations extracted from the main correlation matrix (`corr_matrix`), categorised by type. Each group has been filtered to remove duplicates, mutually exclusive pairs, and low-signal relationships. The goal is to isolate feature interactions that are meaningful for modelling or interpretation.

Pairs are split into the following categories:
- Popularity correlations (with `popularity` as one of the features)
- Metadata correlations (excluding inter-metadata)
- Inter-metadata correlations (e.g. *_avg_popularity*  vs *_song_count* )
- Audio feature correlations (between non-metadata continuous features)
- Categorical one-hot correlations (e.g. genre/year indicators)
- Temporal correlations (derived from `temporal_df`)

These structured correlation groups form the foundation for selecting key relationships explored in later subsections and also highlight redundancy, dominance, or dependency across engineered features.



.

.


####Functions

This block contains all of the helper functions used to filter, clean, and categorise correlation pairs before displaying them. The key idea here is to separate correlation outputs into logical groups (e.g. popularity, metadata, feature-to-feature, etc.) and remove redundant or irrelevant relationships - especially mutually exclusive combinations like genre A <-> genre B.



.

.

.


**Defining Function for Categorising and Filtering by Correlation for a given Correlation Matrix**:

code flattens and categorises a given correlation matrix into 4 sections - highly positively, positively, negatively and highly negatively correlated based on given bounds.

In [ ]:
def correlate_categorise(corr_matrix,high_bound=0.5,low_bound=0.1):
  # Flatten the correlation matrix
  correlated_pairs = corr_matrix.unstack()

  # Convert to DataFrame and reset index
  correlated_pairs = correlated_pairs.reset_index()
  correlated_pairs.columns = ['Feature_1', 'Feature_2', 'Correlation']

  # Remove self-pairs
  correlated_pairs = correlated_pairs[correlated_pairs['Feature_1'] != correlated_pairs['Feature_2']]

  # Remove duplicate pairs (A, B) and (B, A)
  correlated_pairs['sorted_pair'] = correlated_pairs.apply(lambda row: tuple(sorted([row['Feature_1'], row['Feature_2']])), axis=1)
  correlated_pairs = correlated_pairs.drop_duplicates(subset='sorted_pair').drop(columns='sorted_pair')

  # Round correlation values
  correlated_pairs['Correlation'] = correlated_pairs['Correlation'].round(2)

  # Segment into categories
  correlated_categorised = correlated_pairs.copy()

  correlated_categorised.loc[correlated_categorised['Correlation'] >= high_bound, 'category'] = 'very_positive'
  correlated_categorised.loc[(correlated_categorised['Correlation'] >= low_bound) & (correlated_categorised['Correlation'] < high_bound),'category'] = 'positive'
  correlated_categorised.loc[(correlated_categorised['Correlation'] <= -low_bound) & (correlated_categorised['Correlation'] > -high_bound),'category'] = 'negative'
  correlated_categorised.loc[correlated_categorised['Correlation'] <= -high_bound, 'category'] = 'very_negative'

  return correlated_categorised.dropna()


.

.

.

.

.


**Functions for Displaying and Filtering:**

This group of functions handles the filtering, exclusion, and final display of correlation pairs based on logical groupings. These utilities ensure that only meaningful, non-redundant feature pairs are shown, while avoiding pairs that are either mutually exclusive or already explored in a different category.

- `remove_mutually_exclusive(df, ...)`: Filters out any correlation pair where both features belong to the same category (e.g. two genres or two year indicators). It relies on `extract()` to determine which category each feature belongs to, and can be configured to match or exclude specific groupings.

- `extract(col_name, ...)`: Given a feature name, this helper function classifies the feature based on inclusion/exclusion rules. It's designed to support flexible category tagging for things like `*_avg_popularity`, `*_song_count`, and one-hot columns.

- `exclude_pairs_from_table(base_df, exclude_df)`: Removes feature pairs from one correlation table if they already exist in another - useful when ensuring categorical or temporal analyses don't duplicate feature-level pairs.

- `display_pairs(df, display_pair_name)`: Organises a correlation DataFrame into four bins (`very_positive`, `positive`, `negative`, `very_negative`) and prints each one cleanly using the given pair label. This ensures clarity when viewing structured results grouped by context (e.g. popularity, metadata, categorical).

These functions form the backbone of the structured correlation filtering system used throughout Section 2.2.4, allowing flexible group definitions and precise pair curation.


In [ ]:
def remove_mutually_exclusive(df,included_columns=[''],excluded_columns=[''],inter=True):
  df = df.copy()

  df['category_1'] = df['Feature_1'].apply(lambda col_name: extract(col_name, included_columns, excluded_columns, inter))
  df['category_2'] = df['Feature_2'].apply(lambda col_name: extract(col_name, included_columns, excluded_columns, inter))

  df = df[df['category_1'] != df['category_2']]  # exclude pairs within the same category

  return df.drop(columns=['category_1', 'category_2'])


In [ ]:
#extract metadata values
def extract(col_name, included_columns=[''], excluded_columns=[''], inter=True):

  if excluded_columns != ['']:
    if inter == True:
      if col_name in excluded_columns:
        return 'other'
      elif col_name in included_columns:
        return str(col_name)
      else:
        return 'other'
    elif inter == False:
      if col_name in excluded_columns:
        return 'other'
      elif col_name in included_columns:
        return 'included'
      else:
        return 'other'

  if included_columns != ['']:
    if inter == True:
      if col_name in included_columns:
        return str(col_name)
      else:
        return 'other'
    elif inter == False:
      if col_name in included_columns:
        return 'included'
      else:
        return 'other'


In [ ]:
def exclude_pairs_from_table(base_df, exclude_df):

  cols_to_merge = ['Feature_1', 'Feature_2']

  merged_df = pd.merge(base_df, exclude_df[cols_to_merge], how='left', on=cols_to_merge, indicator=True)
  base_df = merged_df[merged_df['_merge'] == 'left_only'].drop('_merge', axis=1)

  return base_df


In [ ]:
def display_pairs(df, display_pair_name):

  very_positive = df[df['category'] == 'very_positive']
  positive = df[df['category'] == 'positive']
  negative = df[df['category'] == 'negative']
  very_negative = df[df['category'] == 'very_negative']

  if len(very_positive) != 0:
    print(f"\nVery Positively Correlated {display_pair_name} Pairs:")
    display(very_positive.sort_values(by='Correlation', ascending=False).drop(columns='category').reset_index(drop=True))

  if len(positive) != 0:
    print(f"\n\n\nPositively Correlated {display_pair_name} Pairs:")
    display(positive.sort_values(by='Correlation', ascending=False).drop(columns='category').reset_index(drop=True))

  if len(negative) != 0:
    print(f"\n\n\nNegatively Correlated {display_pair_name} Pairs:")
    display(negative.sort_values(by='Correlation', ascending=True).drop(columns='category').reset_index(drop=True))

  if len(very_negative) != 0:
    print(f"\n\n\nVery Negatively Correlated {display_pair_name} Pairs:")
    display(very_negative.sort_values(by='Correlation', ascending=True).drop(columns='category').reset_index(drop=True))



.

.

.

.

.


**Function for Scatter-Plotting:**

This function generates a grid of scatter plots for a list of feature pairs, coloured by `popularity`. It's used to quickly explore how pairs of features interact and whether any visible clustering, gradient, or shape exists in relation to popularity. The function supports a custom colour map, and lays out the plots in rows of three for readability. A colour bar is included in each subplot to show the popularity gradient.


In [ ]:
def plot_scatter_grid(df, scatter_pairs, title_suffix='', cmap='viridis'):
    num_plots = len(scatter_pairs)
    cols = 3

    # use ceiling division to get number of rows
    rows = (num_plots + cols - 1) // cols

    fig, axes = plt.subplots(rows, cols, figsize=(18, 5 * rows))
    axes = axes.flatten()

    # plot each scatter pair using simple index-based loop
    for i in range(num_plots):
        x = scatter_pairs[i][0]
        y = scatter_pairs[i][1]

        scatter = axes[i].scatter(df[x], df[y], c=df['popularity'], cmap=cmap)
        axes[i].set_title(f'{x} vs {y} {title_suffix}')
        axes[i].set_xlabel(x)
        axes[i].set_ylabel(y)
        fig.colorbar(scatter, ax=axes[i], label='Popularity')

    # remove unused subplot axes if there are any
    for j in range(num_plots, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()



.

.

.

.

.


####**Popularity Correlation Pairs**

This table isolates all feature pairs where `popularity` is one of the variables. As expected, the strongest correlations are with `artist_avg_popularity` and `genre_avg_popularity`, both of which reflect external metadata influence. We also observe moderate correlations with `year_avg_popularity` and `year_song_count`, indicating a temporal influence, and minor musical feature relationships like `danceability` and `loudness`. Negatively correlated features include `instrumentalness` and `duration_mins`, suggesting that instrumentals and longer tracks tend to underperform in popularity.


In [ ]:
popularity_pairs = remove_mutually_exclusive(correlate_categorise(corr_matrix), included_columns=['popularity'])

In [ ]:
display_pairs(popularity_pairs, display_pair_name='Popularity')


.

.

.

.

.


####**Metadata Correlation Pairs (Excluding Inter-Metadata Pairs)**

This section displays feature pairs involving at least one metadata-derived column (such as `*_avg_popularity` or `*_song_count`), excluding any combinations that are purely inter-metadata (e.g. artist metadata <-> genre metadata). Popularity-based pairs have also been filtered out to avoid duplication with the previous section.

Positive correlations show expected relationships like higher `genre_avg_popularity` aligning with more `danceable` or `valence`-rich tracks, while `acousticness` and `genre_song_count` also trend together, suggesting genre-scale acoustic trends. On the negative side, instrumentals and longer tracks tend to be associated with lower average popularity across both artist and genre groupings. These relationships help identify softer trends that sit between raw audio features and broader artist/genre effects.


In [ ]:
metadata_cols = ['artist_avg_popularity', 'genre_avg_popularity', 'year_avg_popularity', 'artist_song_count', 'genre_song_count', 'year_song_count']
metadata_pairs = remove_mutually_exclusive(correlate_categorise(corr_matrix), included_columns=metadata_cols, inter=False)

In [ ]:
#excluding popularity pair features we have already seen
metadata_pairs = exclude_pairs_from_table(metadata_pairs, popularity_pairs)

In [ ]:
display_pairs(metadata_pairs, display_pair_name='Metadata')


.

.

.

.

.


####**Inter-Metadata Correlation Pairs**

This section isolates correlations between metadata-derived features only - specifically those combining different types of popularity and song count statistics (e.g. artist metadata <-> year metadata). Popularity-based pairs already explored earlier are excluded to avoid redundancy.

Strong correlations include `artist_avg_popularity` to `genre_avg_popularity`, and `year_avg_popularity` to `year_song_count`, reflecting structural overlap between these aggregated metrics. Weaker negative correlations suggest that more prolific artists and genres tend to be associated with lower average popularity, which may point to long-tail behaviour or oversaturation effects.

In [ ]:
metadata_cols = ['artist_avg_popularity', 'genre_avg_popularity', 'year_avg_popularity', 'artist_song_count', 'genre_song_count', 'year_song_count']
intermetadata_pairs = remove_mutually_exclusive(correlate_categorise(corr_matrix), included_columns=metadata_cols, inter=True)

In [ ]:
#excluding popularity pair features we have already seen
intermetadata_pairs = exclude_pairs_from_table(exclude_pairs_from_table(intermetadata_pairs, popularity_pairs), metadata_pairs)

In [ ]:
display_pairs(intermetadata_pairs, display_pair_name='Inter-Metadata')


.

.

.

.

.


####**Feature Correlation Pairs**

This section isolates correlations between non-metadata audio features (e.g. `energy`, `valence`, `loudness`, etc.), excluding any pairs already shown in previous categories. The strongest relationship is between `energy` and `loudness` (r = 0.79), which reflects their shared role in musical intensity.

Other notable positive correlations include `danceability` with `valence`, and `instrumentalness` with `duration_mins`, suggesting that happier tracks tend to be more danceable, and instrumentals are often longer. Strong negative correlations appear between `acousticness` and both `energy` and `loudness`, highlighting a fundamental contrast between high-energy productions and more stripped-back acoustic recordings.

In [ ]:
feat_pairs = correlate_categorise(corr_matrix)

In [ ]:
#excluding popularity pair features we have already seen
feat_pairs = exclude_pairs_from_table(exclude_pairs_from_table(exclude_pairs_from_table(
                feat_pairs, popularity_pairs), metadata_pairs), intermetadata_pairs)

In [ ]:
display_pairs(feat_pairs, display_pair_name='Feature')


.

.

.

.

.


####**Categorical Correlation Pairs**

This section explores meaningful correlations involving categorical one-hot encoded features such as `genre_*`, `year_*`, and musical metadata indicators like `keyOf*`. Columns representing metadata values (e.g. `genre_avg_popularity`, `year_song_count`) are specified to be excluded as they have a similar name to the one-hot columns, e.g. `genre_x`.

High positive correlations include strong genre-popularity links such as `genre_hip-hop`, `genre_dance`, and `genre_pop`, as well as recent years (`year_2022`, `year_2021`) showing elevated average popularity - consistent with the dataset's known recency bias. Negative correlations identify older years and certain niche genres (e.g. `trip-hop`, `piano`) that are underrepresented or consistently less popular.

These correlations reflect broader trends in musical culture and industry shifts over time, and can be useful for genre-specific modelling or for understanding systemic popularity advantages across categories.


In [ ]:
excluded_cols = ['genre_avg_popularity','genre_song_count','year_avg_popularity','year_song_count'] # Note that these columns are excluded from the groupings, not from the pairs entirely -
included_cols = []                                                                                  # it means that these columns are forced to identify as NOT a part of included_cols.
keywords = ['keyOf', 'genre_', 'year_']

#appending a list of all one-hot columns into included_cols[]
for column in train_df:
  for keyword in keywords:
    if keyword in column and column not in excluded_cols:
      included_cols.append(column)

In [ ]:
one_hot_pairs = remove_mutually_exclusive(correlate_categorise(one_hot_corr_matrix, high_bound=0.3, low_bound=0.2),
                                          included_columns=included_cols, excluded_columns=excluded_cols, inter=True)

In [ ]:
display_pairs(one_hot_pairs, display_pair_name='Categorical')


.

.

.

.

.


####**Temporally Respective Correlation Pairs**

This section displays correlations between engineered temporal features derived from `temporal_df`, including `year_decimal`, `running_avg_popularity`, and `running_song_count`. These variables are designed to capture the influence of release timing on popularity in a way that avoids data leakage.

The strongest correlations are between the temporal tracking features themselves, which confirms that the temporal pipeline functions as intended. Additional correlations (e.g. `popularity` <-> `running_avg_popularity`, or `artist_avg_popularity` <-> `year_decimal`) show clear trends over time and reinforce the idea that newer releases and rising popularity are closely linked.

Negative correlations with `duration_mins` and `valence` suggest that emotional tone and track length may shift subtly across time, potentially due to changing production standards or listener preferences.


In [ ]:
temporal_cols = ['year', 'year_decimal', 'artist_avg_popularity', 'genre_avg_popularity', 'year_avg_popularity', 'artist_song_count', 'genre_song_count', 'year_song_count', 'running_avg_popularity', 'running_song_count']

In [ ]:
temporal_pairs = remove_mutually_exclusive(correlate_categorise(temporal_corr_matrix),included_columns=temporal_cols, inter=True)

In [ ]:
#excluding popularity pair features we have already seen
temporal_pairs = exclude_pairs_from_table(exclude_pairs_from_table(exclude_pairs_from_table(
                  temporal_pairs, popularity_pairs), metadata_pairs), intermetadata_pairs)

In [ ]:
display_pairs(temporal_pairs, display_pair_name='Temporal')

**[END]**

.

.

.

.

.

.

.

.

.

.


##**2.3** Visualising Correlated Multivariate Relationships

This section presents a focused set of bivariate scatter plots coloured by popularity to visually explore the relationships between key continuous features. Each scatter plot being coloured by `popularity` offers insight into how certain feature combinations might influence a track's success. All plots are shown in both their raw and `pruned_df` forms - the latter helping to reveal more distinct patterns by removing predictably biased data.

Clear non-linear relationships appear in plots like `energy` vs `loudness`, or `valence` vs `speechiness`, with visibly skewed density and curved boundaries. The pruned version generally shows tighter and more meaningful clustering, suggesting that filtering increases interpretability in a way that aligns with downstream modelling objectives.

Overall, this section acts as a visual precursor to correlation analysis, helping surface feature pairs worth investigating.


.

.

.


###***2.3.1*** - Audio Feature and Popularity Multivariate Scatterplots

These scatter plots includes all of the relevant feature relationships determined in a previous section (**2.2.4**), coloured by popularity for both *df* and *pruned_df* - highlighting the effectiveness of *pruned_df* in revealing patterns amongst the data.

In [ ]:
scatter_pairs = [
    ('energy', 'loudness'), # -----------------VERY POSITIVE

    ('danceability', 'valence'), # ------------POSITIVE
    ('instrumentalness', 'duration_mins'),
    ('energy', 'tempo'),
    ('energy', 'liveness'),
    ('energy', 'speechiness'),
    ('energy', 'valence'),
    ('loudness', 'tempo'),
    ('loudness', 'valence'),
    ('loudness', 'speechiness'),
    ('loudness', 'liveness'),
    ('speechiness', 'liveness'),
    ('acousticness', 'instrumentalness'),

    ('loudness', 'instrumentalness'), # -------NEGATIVE
    ('instrumentalness', 'valence'),
    ('acousticness', 'tempo'),
    ('danceability', 'liveness'),
    ('valence', 'duration_mins'),
    ('speechiness', 'instrumentalness'),
    ('energy', 'instrumentalness'),
    ('speechiness', 'acousticness'),
    ('danceability', 'tempo'),
    ('acousticness', 'liveness'),
    ('acousticness', 'duration_mins'),
    ('speechiness', 'duration_mins'),

    ('energy', 'acousticness'), # -------------VERY NEGATIVE
    ('loudness', 'acousticness')
]


.

.

.


In [ ]:
# Plot using df
plot_scatter_grid(df, scatter_pairs)


.

.

.


In [ ]:
# Plot using pruned_df
plot_scatter_grid(pruned_df, scatter_pairs, title_prefix='(pruned)')


.

.

.

.

.

.

.


###***2.3.2*** - Popularity, Loudness and Energy

This scatter plot visualises the relationship between `loudness` and `energy`, with colour representing `popularity`. The pruned plot (`scale=1.65`) removes points where popularity is heavily explained by artist reputation, revealing a sharp clustering in the upper-right. This aligns with modern production trends where high energy and loud mastering dominate among highly popular songs.


In [ ]:
pruned_df = Prune(df,
                       df['popularity'],
                       df['artist_avg_popularity'],
                       scale=1.65)

In [ ]:
ax1 = df.plot.scatter(x='loudness', y='energy', c='popularity', cmap='viridis')
ax2 = pruned_df.plot.scatter(x='loudness', y='energy', c='popularity', cmap='viridis')


.

.

.


These plots highlight that the most popular and well-performing songs cluster tightly within a specific loudness and energy range - typically loud (around -5 dB) and highly energetic (~0.7). This suggests that modern production trends favour a consistent dynamic profile in hit tracks, where outliers in either direction (too quiet or low-energy) are rarely among the most successful.


In [ ]:
ax1 = pruned_df[pruned_df['popularity'] > 75].plot.scatter(
                                                  x='loudness',
                                                  y='energy',
                                                  c='popularity',
                                                  colormap='viridis')

ax2 = df[df['artist_avg_popularity'] > 60].plot.scatter(
                                                  x='loudness',
                                                  y='energy',
                                                  c='popularity',
                                                  colormap='viridis')

ax3 = df[df['artist_avg_popularity'] > 30].plot.scatter(
                                                  x='loudness',
                                                  y='energy',
                                                  c='popularity',
                                                  colormap='viridis')

ax4 = df[df['artist_avg_popularity'] < 10].plot.scatter(
                                                  x='loudness',
                                                  y='energy',
                                                  c='popularity',
                                                  colormap='viridis')



.

.

.

.

.


###***2.3.3*** - Popularity, Danceability and Valence

This scatter plot explores the relationship between `danceability` and `valence`, with colour indicating `popularity`. After applying the pruning filter (`scale=1.55`) to remove songs whose popularity closely mirrors their artist average, we observe a clearer clustering of higher popularity in the central and upper-right region of the distribution. This suggests that songs which are both happy (valence) and danceable tend to resonate more with listeners (where danceability is the dominant feature), supporting the idea that emotionally upbeat tracks hold broad commercial appeal.


In [ ]:
pruned_df = Prune(df,
                       df['popularity'],
                       df['artist_avg_popularity'],
                       scale=1.55)

In [ ]:
ax1 = df.plot.scatter(x='danceability',
                      y='valence',
                      c='popularity',
                      cmap='viridis')

ax2 = pruned_df.plot.scatter(x='danceability',
                                y='valence',
                                c='popularity',
                                cmap='viridis')


.

.

.



We further investigate the same relationship by filtering the data across different popularity and artist popularity bands. Notably, high popularity songs (`popularity > 60`) continue to cluster in the top-right region, while filtering for artists with high average popularity reveals a similar emotional and danceable signature. Conversely, songs from low-popularity artists exhibit no such concentration, supporting the idea that artist influence plays a major role in driving the emotional accessibility of popular music.


In [ ]:
ax1 = pruned_df[pruned_df['popularity'] > 60].plot.scatter(
                                                  x='danceability',
                                                  y='valence',
                                                  c='popularity',
                                                  colormap='viridis')

ax2 = df[df['artist_avg_popularity'] > 60].plot.scatter(
                                                  x='danceability',
                                                  y='valence',
                                                  c='popularity',
                                                  colormap='viridis')

ax3 = df[df['artist_avg_popularity'] > 30].plot.scatter(
                                                  x='danceability',
                                                  y='valence',
                                                  c='popularity',
                                                  colormap='viridis')

ax4 = df[df['artist_avg_popularity'] < 10].plot.scatter(
                                                  x='danceability',
                                                  y='valence',
                                                  c='popularity',
                                                  colormap='viridis')



.

.

.

.

.


###***2.3.4*** - Popularity, Tempo and Danceability

This scatter plot explores the relationship between `tempo` and `danceability`, with colour indicating `popularity`. Using the pruning filter (`scale=1.55`), we remove popularity values that closely align with the artist averages, revealing a clearer structure. The filtered data shows that highly danceable songs tend to cluster around mid-tempo ranges (roughly 90-140 BPM), suggesting that extreme tempos may hinder danceability and mainstream appeal.


In [ ]:
pruned_df = Prune(df,
                       df['popularity'],
                       df['artist_avg_popularity'],
                       scale=1.55)

In [ ]:
ax1 = df.plot.scatter(x='tempo', y='danceability', c='popularity', cmap='viridis')
ax2 = pruned_df.plot.scatter(x='tempo', y='danceability', c='popularity', cmap='viridis')


.

.

.


Interestingly, when filtering by artist average popularity, we find that the pattern seen in the pruned dataset is broadly replicated-particularly among artists with moderate-to-high popularity. These clusters reinforce the idea that high-performing tracks typically favour a mid-tempo, high-danceability profile. Conversely, artists with low average popularity produce far more scattered results, suggesting less consistent alignment with popular sonic traits.


In [ ]:
ax1 = pruned_df[pruned_df['popularity'] > 70].plot.scatter(
                                                  x='tempo',
                                                  y='danceability',
                                                  c='popularity',
                                                  colormap='viridis')

ax2 = df[df['artist_avg_popularity'] > 60].plot.scatter(
                                                  x='tempo',
                                                  y='danceability',
                                                  c='popularity',
                                                  colormap='viridis')

ax3 = df[df['artist_avg_popularity'] > 30].plot.scatter(
                                                  x='tempo',
                                                  y='danceability',
                                                  c='popularity',
                                                  colormap='viridis')

ax4 = df[df['artist_avg_popularity'] < 10].plot.scatter(
                                                  x='tempo',
                                                  y='danceability',
                                                  c='popularity',
                                                  colormap='viridis')



.

.

.

.

.


###***2.3.5*** - Popularity, Speechiness and Duration(mins)

This scatter plot explores the relationship between `speechiness` and `duration_mins`, coloured by `popularity`. Applying the pruning filter (`scale=1.75`) reveals a clearer trend: higher popularity tracks tend to avoid extremes - neither overly short nor long, and with moderate levels of speechiness. The filter helps isolate this more stable core, suggesting that moderately structured vocal content and song length may contribute to broader appeal.


In [ ]:
pruned_df = Prune(df,
                       df['popularity'],
                       df['artist_avg_popularity'],
                       scale=1.75)

In [ ]:
ax1 = df.plot.scatter(x='speechiness', y='duration_mins', c='popularity', cmap='viridis')
ax2 = pruned_df.plot.scatter(x='speechiness', y='duration_mins', c='popularity', cmap='viridis')


.

.

.


When segmenting by artist popularity, we observe that higher-quality, more popular songs (top) concentrate in a narrower range of moderate speechiness and durations between 2-5 minutes. This aligns with mainstream expectations for pop structures. In contrast, less popular artists (bottom) show wide variance, especially towards extremes-longer, speech-heavy tracks that likely deviate from norms. These plots further support the idea that predictability and structure play a role in achieving mass appeal.


In [ ]:
ax1 = pruned_df[pruned_df['popularity'] > 60].plot.scatter(
                                                  x='speechiness',
                                                  y='duration_mins',
                                                  c='popularity',
                                                  colormap='viridis')

ax2 = df[df['artist_avg_popularity'] > 60].plot.scatter(
                                                  x='speechiness',
                                                  y='duration_mins',
                                                  c='popularity',
                                                  colormap='viridis')

ax3 = df[df['artist_avg_popularity'] > 30].plot.scatter(
                                                  x='speechiness',
                                                  y='duration_mins',
                                                  c='popularity',
                                                  colormap='viridis')

ax4 = df[df['artist_avg_popularity'] < 10].plot.scatter(
                                                  x='speechiness',
                                                  y='duration_mins',
                                                  c='popularity',
                                                  colormap='viridis')



.

.

.

.

.


###***2.3.6*** - Popularity, Energy and Acousticness

This scatter plot explores the relationship between `energy` and `acousticness`, coloured by `popularity`. After applying the pruning filter (`scale=1.75`), a clear inverse pattern emerges - tracks with high energy tend to be less acoustic. The filtered data shows that popular tracks cluster in this low-acousticness/high-energy zone, reinforcing the idea that energetic, studio-produced music typically outperforms acoustic recordings in the mainstream.


In [ ]:
pruned_df = Prune(df,
                       df['popularity'],
                       df['artist_avg_popularity'],
                       scale=1.75)

In [ ]:
ax1 = df.plot.scatter(x='energy',
                      y='acousticness',
                      c='popularity',
                      cmap='viridis')

ax2 = pruned_df.plot.scatter(x='energy',
                                y='acousticness',
                                c='popularity',
                                cmap='viridis')


.

.

.


The filtered scatter plot (popularity > 70) highlights a strong inverse relationship between `energy` and `acousticness`, with most high-performing tracks concentrated in the low-acousticness, high-energy quadrant. Compared to artists with varying average popularity, this cluster is especially pronounced among top performers. These results support the view that high-energy, non-acoustic (i.e., more electronically produced) tracks dominate popular music, while lower-tier artists tend to show greater acoustic variation and less energetic content.


In [ ]:
ax1 = pruned_df[pruned_df['popularity'] > 70].plot.scatter(
                                                  x='energy', y='acousticness', c='popularity',
                                                  colormap='viridis')

ax2 = df[df['artist_avg_popularity'] > 60].plot.scatter(
                                                  x='energy', y='acousticness', c='popularity',
                                                  colormap='viridis')

ax3 = df[df['artist_avg_popularity'] > 30].plot.scatter(
                                                  x='energy', y='acousticness', c='popularity',
                                                  colormap='viridis')

ax4 = df[df['artist_avg_popularity'] < 10].plot.scatter(
                                                  x='energy', y='acousticness', c='popularity',
                                                  colormap='viridis')



.

.

.

.

.


###***2.3.7*** - Popularity, Instrumentalness and Loudness

This scatter plot explores the relationship between `instrumentalness` and `loudness`, with colour indicating `popularity`. After applying the pruning filter (`scale=1.75`), the inverse association becomes more visible: highly instrumental tracks are typically quieter and less popular. The filtered data helps surface this pattern more clearly, suggesting that louder, more vocal-forward songs are favoured in mainstream music.


In [ ]:
pruned_df = Prune(df,
                       df['popularity'],
                       df['artist_avg_popularity'],
                       scale=1.75)

In [ ]:
ax1 = df.plot.scatter(x='instrumentalness',
                      y='loudness',
                      c='popularity',
                      colormap='viridis')

ax2 = pruned_df.plot.scatter(x='instrumentalness',
                      y='loudness',
                      c='popularity',
                      colormap='viridis')


.

.

.


Compared across filtered subsets, the relationship between `instrumentalness` and `loudness` becomes clearer. Tracks by low-popularity artists are consistently quieter and more instrumental, while popular tracks cluster toward low `instrumentalness` and high `loudness`. These filtered views reinforce the notion that instrumental music tends to underperform commercially compared to louder, vocal-forward recordings.

In [ ]:
ax1 = pruned_df[pruned_df['popularity'] > 70].plot.scatter(
                                                  x='instrumentalness',
                                                  y='loudness',
                                                  c='popularity',
                                                  colormap='viridis')

ax2 = df[df['artist_avg_popularity'] > 60].plot.scatter(
                                                  x='instrumentalness',
                                                  y='loudness',
                                                  c='popularity',
                                                  colormap='viridis')

ax3 = df[df['artist_avg_popularity'] > 30].plot.scatter(
                                                  x='instrumentalness',
                                                  y='loudness',
                                                  c='popularity',
                                                  colormap='viridis')

ax4 = df[df['artist_avg_popularity'] < 10].plot.scatter(
                                                  x='instrumentalness',
                                                  y='loudness',
                                                  c='popularity',
                                                  colormap='viridis')


**[END]**

.

.

.

.

.

.

.

.

.

.


##**2.4** Time-Respective Temporal Patterns and Trends

This section explores how key popularity signals and metadata features evolve over time using the enriched `temporal_df`, which captures time-aware statistics such as `artist_avg_popularity`, `genre_avg_popularity`, and `year_avg_popularity`. A `year_decimal` column was introduced to emulate continuous release patterns, enabling smoother and more realistic trend analysis.

The plots in this section illustrate long-term shifts in popularity, artist influence, and output volume, using only information available up to each song's release. This allows us to observe meaningful patterns while preserving causal integrity — forming a basis for cleaner temporal modelling and interpretation.



.

.


###***2.4.1*** - Running Average Popularity Over Time

This plot displays the global running average of song popularity over time, smoothed using a 500-entry rolling window. The clear upward trajectory indicates a general rise in average popularity values across the dataset, potentially reflecting increased catalogue optimisation, algorithmic promotion, or shifts in streaming-era user behaviour.

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(data=temporal_df.sort_values(by='year_decimal').rolling(window=500).mean(), x='year_decimal', y='running_avg_popularity')
plt.title('Running Average Popularity Over Time (Global)')
plt.xlabel('Year (Continuous)')
plt.ylabel('Running Average Popularity')
plt.tight_layout()
plt.show()


.

.

.


###***2.4.2*** - Average Artist Popularity Over Time

This plot shows the running average of `artist_avg_popularity` over time, using a 500-entry rolling mean to smooth fluctuations. The steady upward trend highlights how artist momentum has increased across the streaming era, suggesting that newer releases benefit more consistently from prior success than in earlier years.

Additionally, dips appear at the start of each year - suggesting that some sort of system reset occurs each year that Spotify may utilise to regulate the model.

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(data=temporal_df.sort_values(by='year_decimal').rolling(window=500).mean(), x='year_decimal', y='artist_avg_popularity')
plt.title('Running Artist Average Popularity Over Time')
plt.xlabel('Year (continuous)')
plt.ylabel('Artist Average Popularity (at time of release)')
plt.tight_layout()
plt.show()


.

.

.

.

.


###***2.4.3*** -Average Genre Popularity Over Time

This plot tracks the rolling average of `genre_avg_popularity` over time, based on each song's genre context at release. The long-term upward trend suggests that genre affiliation is increasingly predictive of popularity — potentially reflecting the platform-driven rise of trend-dominant genres and algorithmic reinforcement of high-performing categories.

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(data=temporal_df.sort_values(by='year_decimal').rolling(window=500).mean(), x='year_decimal', y='genre_avg_popularity')
plt.title('Running Genre Average Popularity Over Time')
plt.xlabel('Year (continuous)')
plt.ylabel('Genre Average Popularity (at time of release)')
plt.tight_layout()
plt.show()


.

.

.

.

.


###***2.4.4*** -Average Year Popularity Over Time

This graph visualises the time-respective `year_avg_popularity`, capturing how the average popularity within each release year has evolved over time. The stepped pattern reflects our yearly aggregation logic, with sharp dips and peaks where the mean is unstable at the start of each year, while the overall upward trend suggests that more recent years have consistently higher baseline popularity — potentially driven by streaming platform bias, catalogue expansion, and changing user engagement metrics.

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(data=temporal_df.sort_values(by='year_decimal').rolling(window=500).mean(), x='year_decimal', y='year_avg_popularity')
plt.title('Running Year Average Popularity Over Time')
plt.xlabel('Year (continuous)')
plt.ylabel('Year Average Popularity (at time of release)')
plt.tight_layout()
plt.show()


.

.

.

.

.


###***2.4.5*** - Distribution of Song Count Over Time

This histogram illustrates the yearly distribution of songs within the dataset from 2000 to 2022. The number of releases increases steadily over time, reflecting platform growth and digital accessibility.

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(temporal_df['year'], bins=23)
plt.title('Distribution of Songs Released per Year')
plt.xlabel('Year')
plt.ylabel('Number of Songs')
plt.xticks(range(2000, 2023), rotation=45)
plt.tight_layout()
plt.show()

**[END]**

.

.

.

.

.

.

.

.

.

.


#**Section 3** - Data Analysis using Regression

This section begins the core modelling pipeline. Using the fully preprocessed and normalised dataset `train_df`, we now attempt to predict song `popularity` using a range of regression-based approaches. The goal is to determine how well various models can capture the complex, nonlinear relationships embedded in the music features - from structural audio elements like `loudness` and `danceability`, to engineered metadata like `artist_avg_popularity`.

Rather than focusing on theoretical benchmarking, this section prioritises interpretability, simplicity, and a realistic appraisal of model behaviour in the context of real-world music data. We begin with linear regression as a baseline, then expand into ensemble and deep learning methods later in **Section 4**. Throughout, we maintain consistent evaluation metrics for fair comparison, with additional attention given to feature importance and residual behaviour.


.

.


##**3.1** Instantiation

###***3.1.1*** - Instantiating new DataFrame *reg_df*

In [ ]:
reg_df = train_df.copy()


.

.

.


##**3.2** Linear Regression

This section introduces the first baseline model used in the project: standard linear regression. Linear regression provides a transparent and interpretable starting point for understanding how the selected features relate to song popularity.

The model is trained on the normalised `reg_df` dataset, where `popularity` is the target and all other features serve as inputs. After performing an 80/20 train-test split, the `LinearRegression` model is fitted and evaluated using both the R² score and a custom average prediction accuracy metric.

The resulting R² score of ~**76.2%** suggests a moderately strong linear relationship between features and popularity. Coefficient analysis reveals the weight (β) assigned to each feature, helping to interpret which variables have the most influence. However the custom accuracy measure (based on proportional error) yields ~**63.02%**, displaying very poor actual predictive power.

Although limited by its linear assumptions, this model establishes a performance baseline. It offers a foundation for comparison with more advanced ensemble and deep learning models explored in later sections.


.

.


###***3.2.1*** - Regression

This sets up a basic linear regression model using the normalised `reg_df` dataset. The `popularity` column is assigned as the target variable, and all other features are treated as inputs. The data is split into the training and testing sets (80/20 split), and a `LinearRegression` model is fitted to the training data.


In [ ]:
#assigning target and independant variables
target = 'popularity'


y = reg_df.loc[:, 'popularity']
X = reg_df.drop(labels='popularity', axis=1)

# Splitting to training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#Initiate a regression
linearRegressor = LinearRegression()
#trains the regressor
linearRegressor.fit(X_train, y_train)


.

.

.


###***3.2.2*** - Coefficient Analysis

This step outputs the learned linear regression coefficients for each input feature. Each value represents the weight (β) assigned to a feature, indicating its influence on the predicted popularity. Higher absolute values suggest stronger impact, with positive or negative signs showing direction of effect.

In [ ]:
#outputs the coefficients against each independant variable
display(pd.DataFrame(linearRegressor.coef_, X.columns, columns = ['Coefficient(β)']))


.

.

.


###***3.2.3*** - r² Prediction Score

The R² score is calculated on the test set to evaluate how well the linear regression model explains variance in popularity. The resulting prediction accuracy is ~**76.2%**, indicating a moderately strong linear fit between the features and target variable.

In [ ]:
r2_score = linearRegressor.score(X_test,y_test)
print('\n\nr2 predicted accuracy: ',round(r2_score*100,2),'%\n\n')


.

.


###***3.2.4*** - Predictive Score (Calculated from the Dataset)

To further assess the model's predictive accuracy, each test prediction is compared to its true value. Linear Regression has a relatively poor actual performance of ~**63.02%** accuracy. Shown is a scatter plot of predictions vs. actual values, with the diagonal line `x = y` representing perfect prediction.


In [ ]:
#Calculating predictions from test and training
predictions = linearRegressor.predict(X_test)

#Adding an array of the Actual Values
observations = y_test.to_numpy()


In [ ]:
for x in range(len(predictions)):
  predictions[x] = predictions[x] * 100

for x in range(len(observations)):
  observations[x] = observations[x] * 100

In [ ]:
pred_average = float()
sum_preds = 0

for x in range(len(observations)):

  if predictions[x] < 0:
    predictions[x] = 0
  if predictions[x] > 100:
    prediction[x] = 100


  prediction = predictions[x]
  observation = observations[x]


  if prediction == 0 and observation == 0:
    pred_average = 0

  elif prediction == 0:
    pred_average = (float((prediction/observation)*100))

  elif observation == 0:
    pred_average = (float((observation/prediction)*100))

  elif prediction <= observation:
    pred_average = (float((prediction/observation)*100))

  elif prediction > observation:
    pred_average = (float((observation/prediction)*100))

  sum_preds += pred_average

predAvg = sum_preds/(len(observations))

print("\n\n\nAverage prediction accuracy: "+str(round(predAvg,2))+"%\n\n\n")




#plot actual values against predicted values
plt.plot(observations, predictions, '+')
plt.xlabel('Observation (o)')
plt.ylabel('Prediction (p)')

# plot a line x = y between 0 and 100
x = np.linspace(0, 100, 100)
y = x
plt.plot(x, y)

# --- references ---
#https://seaborn.pydata.org/generated/seaborn.scatterplot.html

**[END]**

.

.

.

.

.

.

.

.

.

.


##**3.3** XGBoost

This section introduces the first ensemble model used in the project - XGBoost. XGBoost is a popular implementation of gradient-boosted decision trees that is well-suited to handling structured data like this. It incrementally builds a strong predictor by combining the outputs of many shallow trees, each correcting the errors of the one before.

The parameters used (n_estimators=300, max_depth=6, learning_rate=0.05, etc.) are chosen to strike a balance between accuracy and overfitting. The model is trained using the same preprocessed dataset (reg_df) as before, and evaluated using R² and a custom prediction accuracy score.

Despite its complexity, the model remains interpretable through feature importance and residual evaluation. This model sets a performance benchmark that can be compared against both simpler linear methods and more flexible deep learning approaches.


.

.


###***3.3.1*** - Regression

This sets up an XGBoost regression model using the normalised `reg_df` dataset. `popularity` is assigned as the target variable, with all other features used as inputs. After splitting into training and testing sets (80/20 split), the `XGBRegressor` is configured with 300 trees, controlled depth, and subsampling to avoid overfitting, then trained on the training data.


In [ ]:
#assigning target and independant variables
target = 'popularity'

y = reg_df.loc[:, target]
X = reg_df.drop(labels=target, axis=1)

# Splitting to training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#Initiate an XGBoost regressor
xgbRegressor = xgb.XGBRegressor(
      objective='reg:squarederror',
      n_estimators=300,         # more trees = more capacity
      max_depth=6,              # controls model complexity
      learning_rate=0.05,       # lower = slower but more accurate learning
      subsample=0.8,            # prevents overfitting
      colsample_bytree=0.8,     # use only subset of features per tree
      n_jobs=-1                 # use all CPU cores
)

#trains the regressor
xgbRegressor.fit(X_train, y_train)


.

.

.


###***3.3.2*** - Feature Importance Analysis

This step outputs the feature importances from the trained XGBoost model. Unlike linear regression coefficients, these values reflect how often and how effectively each feature is used in the decision trees to reduce prediction error. Higher values indicate stronger influence on the predicted popularity.

In [ ]:
# Get feature importances
importances = xgbRegressor.feature_importances_

# Create a DataFrame to display feature importances
display(pd.DataFrame(importances, X.columns, columns=['Feature Importance']))


.

.

.


###***3.3.3*** - r² Prediction Score

The R² score is calculated on the test set to evaluate how well the XGBoost model explains variance in popularity. The resulting prediction accuracy is ~**79.76%**, indicating a much stronger fit and improved performance over linear regression.


In [ ]:
r2_score = xgbRegressor.score(X_test,y_test)
print('\n\nr2 predicted accuracy: ',round(r2_score*100,2),'%\n\n')


.

.


###***3.3.4*** - Predictive Score (Calculated from the Dataset)

To further assess the model's predictive accuracy, each test prediction is compared to its true value. XGBoost has a moderate actual performance of ~**64.86%** accuracy. Shown is a scatter plot of predictions vs. actual values, with the diagonal line `x = y` representing perfect prediction.

In [ ]:
#Calculating predictions from test and training
predictions = xgbRegressor.predict(X_test)

#Adding an array of the Actual Values
observations = y_test.to_numpy()

In [ ]:
for x in range(len(predictions)):
  predictions[x] = predictions[x] * 100

for x in range(len(observations)):
  observations[x] = observations[x] * 100

In [ ]:
pred_average = float()
sum_preds = 0

for x in range(len(observations)):

  if predictions[x] < 0:
    predictions[x] = 0
  if predictions[x] > 100:
    prediction[x] = 100


  prediction = predictions[x]
  observation = observations[x]


  if prediction == 0 and observation == 0:
    pred_average = 0

  elif prediction == 0:
    pred_average = (float((prediction/observation)*100))

  elif observation == 0:
    pred_average = (float((observation/prediction)*100))

  elif prediction <= observation:
    pred_average = (float((prediction/observation)*100))

  elif prediction > observation:
    pred_average = (float((observation/prediction)*100))

  sum_preds += pred_average

predAvg = sum_preds/(len(observations))

print("\n\n\nAverage prediction accuracy: "+str(round(predAvg,2))+"%\n\n\n")




#plot actual values against predicted values
plt.plot(observations, predictions, '+')
plt.xlabel('Observation (o)')
plt.ylabel('Prediction (p)')

# plot a line x = y between 0 and 100
x = np.linspace(0, 100, 100)
y = x
plt.plot(x, y)

# --- references ---
#https://seaborn.pydata.org/generated/seaborn.scatterplot.html

**[END]**

.

.

.

.

.

.

.

.

.

.


##**3.4** Random Forest

This section applies a Random Forest Regressor, the first bagging-based ensemble model in the project. Random Forests combine the predictions of multiple decision trees trained on random subsets of both the data and features. This ensemble approach helps reduce variance and avoid overfitting, making it well-suited for noisy, structured datasets like this one.

The model is trained on the normalised `reg_df` dataset, with `popularity` as the target variable. Using 60 estimators and full CPU parallelism, the regressor captures complex feature interactions while maintaining strong generalisation. Performance is evaluated using both the R² score and the custom prediction accuracy metric.

With an R² score of ~**81.91%** and an average proportional accuracy of ~**66.21%**, this model currently offers the strongest performance so far. Feature importances are extracted to understand which variables were most influential in predicting song popularity.


.

.


###***3.4.1*** - Regression

This sets up a Random Forest regression model using the normalised `reg_df` dataset. The target variable is `popularity`, with all remaining features treated as inputs. After an 80/20 train-test split, the `RandomForestRegressor` is configured with 60 trees and parallel processing enabled. The model is then trained on the training set.

In [ ]:
#assigning target and independant variables
target = 'popularity'

y = reg_df.loc[:, target]
X = reg_df.drop(labels=target, axis=1)

# Splitting to training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#Initiate a Random Forest Regressor
rfRegressor = RandomForestRegressor(n_estimators=60, random_state=0, n_jobs=-1)

#train the regressor
rfRegressor.fit(X_train, y_train)

# about +5m 20s compute time per 30 estimators


.

.

.


###***3.4.2*** - Feature Importance Analysis

This step outputs the feature importances from the trained Random Forest model. Unlike linear regression coefficients, these values reflect how often and how effectively each feature is used in the decision trees to reduce prediction error. Higher values indicate stronger influence on the predicted popularity.

In [ ]:
# Get feature importances instead of coefficients
importances = rfRegressor.feature_importances_

# Create a DataFrame to display feature importances
display(pd.DataFrame(importances, X.columns, columns=['Feature Importance']))


.

.

.


###***3.4.3*** - r² Prediction Score

The R² score is calculated on the test set to evaluate how well the Random Forest model explains variance in popularity. The resulting prediction accuracy is ~**81.91%**, indicating a strong fit and the highest performance so far, surpassing both the linear and XGBoost regressors.


In [ ]:
r2_score = rfRegressor.score(X_test,y_test)
print('\n\nr2 predicted accuracy: ',round(r2_score*100,2),'%\n\n')


.

.


###***3.4.4*** - Predictive Score (Calculated from the Dataset)

To further assess the model's predictive accuracy, each test prediction is compared to its true value. Random Forest has a moderate actual performance of ~**66.21%** accuracy. Shown is a scatter plot of predictions vs. actual values, with the diagonal line `x = y` representing perfect prediction.


In [ ]:
#Calculating predictions from test and training
predictions = rfRegressor.predict(X_test)

#Adding an array of the Actual Values
observations = y_test.to_numpy()

In [ ]:
for x in range(len(predictions)):
  predictions[x] = predictions[x] * 100

for x in range(len(observations)):
  observations[x] = observations[x] * 100

In [ ]:
pred_average = float()
sum_preds = 0

for x in range(len(observations)):

  if predictions[x] < 0:
    predictions[x] = 0
  if predictions[x] > 100:
    prediction[x] = 100


  prediction = predictions[x]
  observation = observations[x]


  if prediction == 0 and observation == 0:
    pred_average = 0

  elif prediction == 0:
    pred_average = (float((prediction/observation)*100))

  elif observation == 0:
    pred_average = (float((observation/prediction)*100))

  elif prediction <= observation:
    pred_average = (float((prediction/observation)*100))

  elif prediction > observation:
    pred_average = (float((observation/prediction)*100))

  sum_preds += pred_average

predAvg = sum_preds/(len(observations))

print("\n\n\nAverage prediction accuracy: "+str(round(predAvg,2))+"%\n\n\n")




#plot actual values against predicted values
plt.plot(observations, predictions, '+')
plt.xlabel('Observation (o)')
plt.ylabel('Prediction (p)')

# plot a line x = y between 0 and 100
x = np.linspace(0, 100, 100)
y = x
plt.plot(x, y)

# --- references ---
#https://seaborn.pydata.org/generated/seaborn.scatterplot.html

**[END]**

.

.

.

.

.

.

.

.

.

.


#**Section 4** - Deep Learning

This section introduces the deep learning phase of the project. Building on the fully preprocessed and normalised `train_df`, we construct, train, and evaluate a neural network for predicting song popularity. This model represents the most flexible and expressive approach used so far, designed to uncover non-linear and higher-order interactions among features that may not be captured by simpler regression or ensemble methods.

The architecture is built using TensorFlow's `Sequential` API. It includes three double hidden dense layers (256, 128, and 64 units), each using ReLU activation and dropout (rate = 0.2) between each pair to prevent overfitting. A single linear output node is used to predict the final popularity score, allowing for unconstrained regression output. The model is compiled using the RMSprop optimiser and trained using MSE loss.

This neural model establishes the most advanced benchmark in the project. While less interpretable than linear or tree-based models, it reveals more subtle patterns in the data and offers the strongest potential for future prediction accuracy improvements in music analytics.



.

.


In [ ]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Concatenate, Activation, Lambda


.

.

.


##**4.1**  Training *model*

###***4.1.1*** - Fabricating the Model

This subsection defines a deep neural network using TensorFlow's `Sequential` API. The input layer expects 114 features from the one-hot encoded and normalised `train_df`. The model includes 6 hidden layers (with 2x 256, 2x 128, and 2x 64 neurons respectively), each using ReLU activation and dropout regularisation (rate = 0.2) to prevent overfitting. The final output layer uses a linear activation to predict the continuous popularity score.

In [ ]:
X = train_df.drop(labels='popularity', axis=1)

y = train_df['popularity']


#Train and Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, shuffle=True, random_state=0)


.

.

.


Defining Model Architecture:

In [ ]:
excluded_cols = ['artist_avg_popularity', 'genre_avg_popularity', 'year_avg_popularity',
                 'artist_song_count', 'genre_song_count', 'year_song_count']

# Define column groups
one_hot_cols = [col for col in train_df.columns if col.startswith(('genre_', 'year_', 'major', 'minor', 'keyOf', 'time_signature'))
                and col not in excluded_cols]

continuous_cols = [col for col in train_df.columns if col not in one_hot_cols + ['popularity']]

In [ ]:
# Find slice bounds
one_hot_start = train_df.columns.get_loc(one_hot_cols[0])
one_hot_end = train_df.columns.get_loc(one_hot_cols[-1]) + 1

cont_start = train_df.columns.get_loc(continuous_cols[0])
cont_end = train_df.columns.get_loc(continuous_cols[-1]) + 1

In [ ]:
# MODEL ARCHITECTURE

# Input layer (all features)
input_layer = Input(shape=(114,))

# Lambda slices for one-hot and continuous blocks
one_hot_input = Lambda(lambda x: x[:, one_hot_start:one_hot_end])(input_layer)
continuous_input = Lambda(lambda x: x[:, cont_start:cont_end])(input_layer)

# One-hot branch
one_hot_branch = Dense(128)(one_hot_input)
one_hot_branch = BatchNormalization()(one_hot_branch)
one_hot_branch = Activation('relu')(one_hot_branch)
one_hot_branch = Dropout(0.1)(one_hot_branch)

# Continuous branch
cont_branch = Dense(256)(continuous_input)
cont_branch = BatchNormalization()(cont_branch)
cont_branch = Activation('relu')(cont_branch)
cont_branch = Dropout(0.2)(cont_branch)

cont_branch = Dense(128)(cont_branch)
cont_branch = BatchNormalization()(cont_branch)
cont_branch = Activation('relu')(cont_branch)
cont_branch = Dropout(0.2)(cont_branch)

cont_branch = Dense(64)(cont_branch)
cont_branch = BatchNormalization()(cont_branch)
cont_branch = Activation('relu')(cont_branch)
cont_branch = Dropout(0.2)(cont_branch)

# Merge both branches
merged = Concatenate()([one_hot_branch, cont_branch])
merged = Dense(64)(merged)
merged = BatchNormalization()(merged)
merged = Activation('relu')(merged)
merged = Dropout(0.2)(merged)

# Final output
output = Dense(1, activation='linear')(merged)


model = Model(inputs=input_layer, outputs=output)



.

.


###***4.1.2*** - Compilation, Optimisation and Loss Function

The model is compiled using the RMSprop optimiser and mean squared error (MSE) as the loss function. RMSprop is well-suited for handling noisy or sparse data, while MSE is appropriate for continuous regression tasks like predicting popularity.

In [ ]:
model.compile(optimizer='rmsprop',loss='mean_squared_error')


.

.

.


###***4.1.3*** - Training from the Dataset

The model is trained for 30 epochs with a batch size of 96, using 20% of the training data for validation. Training stabilises quickly, with validation loss converging around **0.0053**. After training, the model is evaluated on the test set, yielding a final test loss of **0.0057**, confirming consistent performance. The trained model is saved for future use as `'model.keras'`.

In [ ]:
training = model.fit(X_train, y_train, batch_size=96, shuffle=True, validation_split=0.2, epochs=15)
model.evaluate(X_test, y_test)


.

.


In [ ]:
model.save('model.keras')

**[END]**

.

.

.

.

.

.

.

.

.

.


##**4.2**  Analysis of  *model*

###***4.2.1*** - Model Summary

This structure provides enough capacity to capture complex patterns while maintaining interpretability and regularisation.

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model_diagram.png', show_shapes=True, show_layer_names=True)


.

.

.


###***4.2.2*** - Training Analysis

This plot shows the training and validation loss curves over the 15 epochs. Both curves steadily decline and closely follow one another, suggesting consistent learning without overfitting. The gap between them remains small, indicating that the model generalises well across both seen and unseen data during training.

In [ ]:
# Plot training & validation loss
plt.figure(figsize=(10, 5))
plt.plot(training.history['loss'], label='Training Loss')
plt.plot(training.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.show()

**[END]**

.

.

.

.

.

.

.

.

.

.


##**4.3** Testing *model*

###***4.3.1*** - Generating Predictions

This code loads observed values and generates predicted values from the trained model using the test set. Both the predictions and true observations are then descaled by multiplying by 100 to return them to their original 0-100 popularity range.

In [ ]:
predictions = model.predict(X_test).flatten() * 100
observations = y_test.to_numpy() * 100


.

.


###***4.3.2*** - Residual Plot

This residual plot visualises the difference between actual popularity and predicted popularity from the deep learning model. Each point represents a test sample, with the **x-axis** showing the true (actual) popularity and the **y-axis** showing the residual (i.e. `Actual - Predicted`). A residual of **0** (highlighted by the red line) indicates a perfect prediction. Points **above** the line represent **under-predictions** (model predicted too low), while points **below** the line indicate **over-predictions** (model predicted too high).


In [ ]:
# Residuals
residuals = observations - predictions

plt.figure(figsize=(10, 5))
plt.scatter(observations, residuals)
plt.axhline(0, color='red')
plt.title('Residual Plot')
plt.xlabel('Actual Popularity')
plt.ylabel('Residual (Observed - Predicted)')
plt.tight_layout()
plt.show()


.

.

.

.

.


###***4.3.3*** - Visualising Distribution Accuracy

This plot compares the distribution of predicted popularity values to the true values in the test set. While the overall shape of the predicted distribution closely follows the actual data, the model slightly overestimates in the mid-range, and slightly underestimates in the high-range. The high concentration of low-popularity tracks is preserved, suggesting that the model captures the overall skew in the dataset reasonably well.


In [ ]:
plt.figure(figsize=(10, 5))
sns.kdeplot(observations, label='Observations', fill=True, clip=(0, 100))
sns.kdeplot(predictions, label='Predictions', fill=True, clip=(0, 100))
plt.title('Distribution of Predicted vs Actual Popularity')
plt.xlabel('Popularity')
plt.legend()
plt.tight_layout()
plt.show()


.

.

.


###***4.3.4*** - r² Prediction Score

The R² score is calculated on the test set to evaluate how well the deep learning model explains variance in popularity. The resulting prediction accuracy is ~**79.86%**, indicating almost the weakest R² performance in the pipeline, albeit a clear improvement over the linear regressor.

In [ ]:
from sklearn.metrics import r2_score

r2 = r2_score(observations, predictions)


print("R-squared predicted accuracy:", str(round((r2*100),2)) +"%")


.

.


###***4.3.5*** - *model* Prediction Accuracy

To further assess the model's predictive accuracy, each test prediction is compared to its true value using a custom proportional closeness metric. This gives an average accuracy score of ~**64.86%**, indicating moderate real-world predictive performance.

Shown is a scatter plot of predictions vs. actual values, with the diagonal line `x = y` representing perfect prediction. The tighter the clustering around this line, the more accurate the model's outputs — though here we observe some consistent underestimation, especially at higher popularity levels.


In [ ]:
pred_average = float()
sum_preds = 0

for x in range(len(observations)):

  if predictions[x] < 0:
    predictions[x] = 0
  if predictions[x] > 100:
    prediction[x] = 100


  prediction = predictions[x]
  observation = observations[x]


  if prediction == 0 and observation == 0:
    pred_average = 0

  elif prediction == 0:
    pred_average = (float((prediction/observation)*100))

  elif observation == 0:
    pred_average = (float((observation/prediction)*100))

  elif prediction <= observation:
    pred_average = (float((prediction/observation)*100))

  elif prediction > observation:
    pred_average = (float((observation/prediction)*100))

  sum_preds += pred_average

predAvg = sum_preds/(len(observations))

print("\n\n\nAverage prediction accuracy: "+str(round(predAvg,2))+"%\n\n\n")




#plot actual values against predicted values
plt.plot(observations, predictions, '+')
plt.xlabel('Observation (o)')
plt.ylabel('Prediction (p)')

# plot a line x = y between 0 and 100
x = np.linspace(0, 100, 100)
y = x
plt.plot(x, y)

# --- references ---
#https://seaborn.pydata.org/generated/seaborn.scatterplot.html

**[END]**

.

.

.

.

.

.

.

.

.

.


##**4.4** Prediction Baseline Performance Sweep

###***4.4.1*** - Defining Functions and Variables

####**Functions**

Defining One-Hot Column Collapser Function *collapse_one_hot_columns()*:

In [ ]:
def collapse_one_hot_columns(df, prefix, new_col_name):
    one_hot_cols = []
    excluded_cols = ['artist_avg_popularity', 'genre_avg_popularity', 'year_avg_popularity',
                     'artist_song_count', 'genre_song_count', 'year_song_count']

    col_list = list((df.drop(columns=excluded_cols)).columns)

    for col in col_list:
        if col.startswith(prefix):
            one_hot_cols.append(col)

    if len(one_hot_cols) == 0:
        return df  # nothing to collapse

    max_vals = df[one_hot_cols].idxmax(axis=1)

    cleaned_vals = []
    for val in max_vals:
        cleaned_vals.append(val.replace(prefix, ''))

    df[new_col_name] = cleaned_vals

    df = df.drop(columns=one_hot_cols)
    return df

Defining Scaler Functions *scale_inputs()* and *unscale_inputs()*:

In [ ]:
def scale_inputs(inputs):
    scaled = inputs.copy()

    # Manual scaling
    if 'loudness' in scaled:
        scaled['loudness'] = (inputs['loudness'] + 60) / 60

    if 'tempo' in scaled:
        scaled['tempo'] = (inputs['tempo'] - 50) / (220 - 50)

    if 'duration_mins' in scaled:
        scaled['duration_mins'] = inputs['duration_mins'] / 20

    if 'artist_avg_popularity' in scaled:
        scaled['artist_avg_popularity'] = inputs['artist_avg_popularity'] / 100

    if 'genre_avg_popularity' in scaled:
        scaled['genre_avg_popularity'] = inputs['genre_avg_popularity'] / 100

    if 'year_avg_popularity' in scaled:
        scaled['year_avg_popularity'] = inputs['year_avg_popularity'] / 100

    if 'artist_song_count' in scaled:
        scaled['artist_song_count'] = inputs['artist_song_count'] / 3000

    if 'genre_song_count' in scaled:
        scaled['genre_song_count'] = inputs['genre_song_count'] / 30000

    if 'year_song_count' in scaled:
        scaled['year_song_count'] = inputs['year_song_count'] / 50000

    return scaled

In [ ]:
def unscale_inputs(inputs, column_names):
    inputs = inputs.copy()

    for i in range(len(column_names)):
        name = column_names[i]

        # Manual unscaling
        if name == 'loudness':
            inputs[i] = (inputs[i] * 60) - 60

        elif name == 'tempo':
            inputs[i] = (inputs[i] * (220-50)) + 50

        elif name == 'duration_mins':
            inputs[i] = inputs[i] * 20

        elif name == 'artist_avg_popularity':
            inputs[i] = inputs[i] * 100

        elif name == 'genre_avg_popularity':
            inputs[i] = inputs[i] * 100

        elif name == 'year_avg_popularity':
            inputs[i] = inputs[i] * 100

        elif name == 'artist_song_count':
            inputs[i] = inputs[i] * 3000

        elif name == 'genre_song_count':
            inputs[i] = inputs[i] * 30000

        elif name == 'year_song_count':
            inputs[i] = inputs[i] * 50000

    return inputs



.

.


Defining Table Display Preparation *prepare_display_df()* Function:

In [ ]:
def prepare_display_df(df):
    # Make a copy so we don't touch the original
    df_disp = df.copy()

    # Columns to be dropped later
    exclude_columns = [
        'genre_song_count',
        'genre_avg_popularity',
        'year_song_count',
        'year_avg_popularity'
    ]

    # Pull out predicted_popularity if it's there
    pred_pop = None
    column_list = list(df_disp.columns)
    for column_name in column_list:
        if column_name == 'predicted_popularity':
            pred_pop = df_disp['predicted_popularity']
            df_disp = df_disp.drop(columns=['predicted_popularity'])
            break

    # Collapse one-hot encoded groups into single columns
    df_disp = collapse_one_hot_columns(df_disp, 'year_', 'year')
    df_disp = collapse_one_hot_columns(df_disp, 'genre_', 'genre')
    df_disp = collapse_one_hot_columns(df_disp, 'keyOf', 'key')
    df_disp = collapse_one_hot_columns(df_disp, 'time_signature_', 'time_signature')

    # Handle mode (major/minor)
    mode_columns = list(df_disp.columns)
    found_major = False
    found_minor = False
    for name in mode_columns:
        if name == 'major':
            found_major = True
        elif name == 'minor':
            found_minor = True
    if found_major and found_minor:
        df_disp['mode'] = df_disp[['major', 'minor']].idxmax(axis=1)
        df_disp = df_disp.drop(columns=['major', 'minor'])

    # Drop any unwanted columns that exist
    final_columns = list(df_disp.columns)
    for column in exclude_columns:
        for existing_column in final_columns:
            if column == existing_column:
                df_disp = df_disp.drop(columns=[column])
                break

    # Put predicted_popularity back at the end if we had it earlier
    if pred_pop is not None:
        df_disp['predicted_popularity'] = pred_pop.round(2)

    return df_disp


.

.

.

.

.


####**Variables**

Feature Groups:

In [ ]:
cont_columns = [
    'danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
    'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_mins'
]

In [ ]:
artist_col = 'artist_avg_popularity'
year_columns = []
genre_columns = []
key_columns = []


for column in X_train.columns:
  if column.startswith('year_'):
    year_columns.append(column)
  elif column.startswith('genre_'):
    genre_columns.append(column)
  elif column.startswith('keyOf'):
    key_columns.append(column)

Scaling:

In [ ]:
# Scale the continuous input columns from X_train using manual scaling function
scaled = scale_inputs(X_train[cont_columns])

# Create an empty list to store the index positions of each continuous feature
index_list = []

# Loop through each column in cont_columns and get its index in X_train
for column in cont_columns:
    col_index = X_train.columns.get_loc(column)
    index_list.append(col_index)

#https://pandas.pydata.org/pandas-docs/version/0.25.0/reference/api/pandas.DataFrame.T.html

In [ ]:
scaled.head()


.

.

.

.

.


###***4.4.2*** - Producing a Baseline Performance Sweep

This subsection generates a baseline performance sweep by varying `artist_avg_popularity` from 0 to 100 in 0.1 steps. For each value, a random song from the training data is sampled, and its `artist_avg_popularity` is overridden while keeping all other features fixed.

The trained model predicts popularity for each modified input, and the result is stored alongside the input features in a new DataFrame. This allows us to observe how the model's predictions scale with increasing artist popularity in isolation.

The resulting line plot shows a clear upward trend, verifying that the model strongly relies on `artist_avg_popularity` as a predictive feature. Noise and fluctuation are more visible at lower values, likely due to more unpredictable song-level deviations among less popular artists.


In [ ]:
rows = []

step_size = 0.1

In [ ]:
# Loop to sweep through values from 0 to 100 for artist_avg_popularity using the given step size
for original_val in tqdm(np.arange(0, 100.1, step_size), desc="Predicting... "):

    # Scale artist_avg_popularity to 0-1
    scaled_val = original_val / 100.0

    # Take a random row from the training set
    x = X_train.sample(1).iloc[0].values.copy()

    # Override artist_avg_popularity
    x[X_train.columns.get_loc(artist_col)] = scaled_val
    #print("\n\nx:",x.round(2))
    # Run prediction
    y_pred = model.predict(x.reshape(1, -1), verbose=False)[0][0]
    y_pred = np.clip(y_pred * 100, 0, 100)

    # Manually unscale all features
    x = unscale_inputs(x, X_train.columns)
    #print("\n\nx unscaled:",x.round(2))
    # Save result as dictionary
    out_row = {}
    for i in range(len(X_train.columns)):
        out_row[X_train.columns[i]] = x[i]

    out_row[artist_col] = round(original_val, 2)
    out_row['predicted_popularity'] = round(y_pred, 2)
    rows.append(out_row)

In [ ]:
# Turn results into a DataFrame
baseline = pd.DataFrame(rows).round(2)

Display:

In [ ]:
print("Random Predicted Popularity Sweep (0.1 Steps)")
display(prepare_display_df(baseline))


plt.figure(figsize=(10, 5))
plt.plot(baseline[artist_col], baseline['predicted_popularity'], linewidth=1)
plt.xlabel('Artist Average Popularity (0-100)')
plt.ylabel('Predicted Popularity (0-100)')
plt.title('Random Predicted Popularity Sweep (0.1 Steps)')
plt.grid(True)
plt.show()

**[END]**

.

.

.

.

.

.

.

.

.

.


#**Section 5** - User Interface

A tool for **ergonomically** predicting a given songs' popularity

##**5.1** Instantiation

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from functools import partial

In [ ]:
new_df = df.copy()
new_df['genre'] = categorical_df['genre']
new_df['year'] = categorical_df['year']

**[END]**

.

.

.

.

.

.

.

.

.

.


##**5.2** User Interface

###***5.1.1*** - Widget Setup

####**Functions**

**Defining function *build_feature_section()*** to create and format a feature input section using sliders for each continuous feature:

In [ ]:
# Layout sliders in two vertical columns
def build_feature_section(sliders):
  midpoint = len(sliders) // 2
  slider_section = widgets.HBox([
      widgets.VBox(sliders[:midpoint]),
      widgets.VBox(sliders[midpoint:])])

  return slider_section


.

.


In [ ]:
# function to create sliders for selected features with realistic default ranges
def create_sliders(features):

    inputs = {}     # stores sliders by feature name
    sliders = []    # stores widgets for layout

    # go through each feature we want to create a slider for
    for feature_name in features:

        # set custom ranges for loudness
        if feature_name == 'loudness':
            fmin = -60       # lowest expected dB
            fmax = 0         # upper bound for dB scale
            label = 'Loudness (dB)'
            default = -20    # somewhere in the middle

        # set custom ranges for tempo
        elif feature_name == 'tempo':
            fmin = 50
            fmax = 220
            label = 'Tempo (BPM)'
            default = 120    # reasonable tempo baseline

        # set custom ranges for duration in minutes
        elif feature_name == 'duration_mins':
            fmin = 0
            fmax = 20
            label = 'Duration (min)'
            default = 3.5    # reasonable track length

        else:
            # default to 0–1 for all other already-normalised features
            fmin = 0
            fmax = 1
            label = feature_name
            default = 0.5

        # create the FloatSlider for current feature
        slider = widgets.FloatSlider(
            description=label,
            min=fmin,
            max=fmax,
            step=round((fmax - fmin) / 100, 3),  # make the step size 1% of the range
            value=default,
            style={'description_width': '180px'},
            layout=widgets.Layout(width='400px')
        )

        inputs[feature_name] = slider   # store for later use
        sliders.append(widgets.HBox([slider]))  # wrap in HBox for layout

    return inputs, sliders



.

.

.


**Defining function *build_artist_section*** for building the seperate input sliders for the important artist-level inputs:

In [ ]:
def build_artist_section():
  inputs = {}     # store sliders by name and info

  # create artist input widgets
  aap_slider = widgets.FloatSlider(
      description='Artist Avg. Popularity',
      min=0,
      max=100,
      step=0.001,
      value=50,
      style={'description_width': '120px'},
      layout=widgets.Layout(width='500px')
  )

  song_count_slider = widgets.BoundedIntText(
      description='Artist Song Count',
      value=100,
      min=0,
      max=3000,
      style={'description_width': '120px'},
      layout=widgets.Layout(width='500px')
  )

  artist_section = widgets.HBox([aap_slider, song_count_slider])
  inputs['artist_avg_popularity'] = aap_slider   # store for later use
  inputs['artist_song_count'] = song_count_slider   # store for later use

  return inputs, artist_section


.

.

.


**Defining function *build_dropdown_section*** for building the dropdown menus for categorical inputs:

In [ ]:
def build_dropdown_section():
  inputs = {}

  genre_input, genre_dropdown = create_dropdown(genre_columns, 'Genre')
  year_input, year_dropdown = create_dropdown(year_columns, 'Year')
  key_input, key_dropdown = create_dropdown(key_columns, 'Key')
  mode_input, mode_dropdown = create_dropdown(mode_columns, 'Mode')
  time_sig_input, time_sig_dropdown = create_dropdown(time_sig_columns, 'Time Signature', default=1)

  dropdown_section = widgets.VBox([
      genre_dropdown,
      year_dropdown,
      key_dropdown,
      mode_dropdown,
      time_sig_dropdown])

  inputs['genre_input'] = genre_input
  inputs['year_input'] = year_input
  inputs['key_input'] = key_input
  inputs['mode_input'] = mode_input
  inputs['time_sig_input'] = time_sig_input

  return inputs, dropdown_section


.

.

.


In [ ]:
# Dropdown helper function
def clean_label(col):
    return (col.replace('genre_', '').replace('year_', '').replace('keyOf', '')
           ).replace('time_signature_0', '0').replace('time_signature_14','1=1').replace('time_signature_24','2/4'
           ).replace('time_signature_34','3/4').replace('time_signature_44','4/4').replace('time_signature_54','5/4')


In [ ]:
# helper function to format a dropdown widget from a set of one-hot encoded columns
def format_dropdown(column_group, index):

  options = []

  # go through all columns in the group and
  # clean up the label if it's not excluded

  for col in column_group:
    label = clean_label(col)
    options.append(label)

  # create a dropdown with the cleaned labels, defaulting to whatever index we pass
  dropdown = widgets.Dropdown(options=options, value=options[index])

  # building a reverse map (essentially a lookup dictionary for translating between formatted UI
  # variable names and actual variable names) for converting cleaned labels back to original column
  reverse_map = {}
  for col in column_group:
      label = clean_label(col)
      reverse_map[label] = col

  return dropdown, reverse_map


.

.

.


In [ ]:
#Organise dropdown layout for clarity
def create_dropdown(columns, label, default=0):
  dropdown, reverse_map = format_dropdown(columns, default)

  return dropdown, widgets.HBox([widgets.Label(f"{label}:", layout=widgets.Layout(width="120px")), dropdown])


.

.

.

.

.


**Assembly**

In [ ]:
columns = [
      'danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
      'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_mins']

feature_inputs, feature_sliders = create_sliders(columns)
slider_section = build_feature_section(feature_sliders)

In [ ]:
artist_inputs, artist_section = build_artist_section()


.

.


Collecting Categorical Columns:

In [ ]:
# loop through all column names in X_train and collect genre columns
excluded_cols = ['artist_avg_popularity', 'genre_avg_popularity', 'year_avg_popularity',
                 'artist_song_count', 'genre_song_count', 'year_song_count']


genre_columns = []
for column in X_train.columns:
  if column.startswith('genre_') and column not in excluded_cols:
    genre_columns.append(column)

# same as above, but for year columns
year_columns = []
for column in X_train.columns :
  if column.startswith('year_') and column not in excluded_cols:
    year_columns.append(column)

# extract key columns that start with 'keyOf'
key_columns = []
for column in X_train.columns:
  if column.startswith('keyOf'):
    key_columns.append(column)

# mode columns are just major and minor, so we can grab those directly
mode_columns = X_train[['major', 'minor']]

# get all time signature columns
time_sig_columns = []
for column in X_train.columns:
  if column.startswith('time_signature'):
    time_sig_columns.append(column)


In [ ]:
dropdown_inputs, dropdown_section = build_dropdown_section()


.

.


Creating *predict_button*:

In [ ]:
# Buttons and output
predict_button = widgets.Button(description='Predict Popularity', button_style='success')
out = widgets.Output()

Assembling the UI:

In [ ]:
# Assemble full UI layout
ui = widgets.VBox([
    slider_section,
    artist_section,
    dropdown_section,
    widgets.HBox([predict_button]),
    out
])

**[END]**

.

.

.

.

.

.

.

.

.

.


##**5.3** Predict Button Logic

###***5.3.1*** - Defining on-button-click *on_predict()* Function

In [ ]:
def on_predict(b):
  # clear previous output in widget box
  with out:
    clear_output()

    # run helper function to get the prediction (already normalised & ready)
    raw_pred = get_prediction()

    # display prediction results
    #print(f"\nRaw model output (0-1): {raw_pred:.6f}")
    print(f"Predicted Popularity: {np.clip(raw_pred * 100, 0, 100):.2f}")


###***5.3.2*** - Defining *get_prediction()* Function

In [ ]:
def get_prediction():
  #print(feature_inputs)
  #print("\n\n",artist_inputs)
  #print("\n\n",dropdown_inputs)

  genre_to_avg_pop = new_df.groupby('genre')['popularity'].mean().to_dict()
  genre_to_song_count = new_df.groupby('genre')['popularity'].count().to_dict()
  year_to_avg_pop = new_df.groupby('year')['popularity'].mean().to_dict()
  year_to_song_count = new_df.groupby('year')['popularity'].count().to_dict()

  user_vals = {}            # One-hot encoded features
  input_vals_raw = {}       # Raw numeric and metadata inputs

  # Fixed continuous inputs
  for c, slider in feature_inputs.items():
    input_vals_raw[c] = slider.value

  # Artist-level inputs
  aap_input = artist_inputs['artist_avg_popularity']
  song_count_input = artist_inputs['artist_song_count']

  input_vals_raw['artist_avg_popularity'] = aap_input.value
  input_vals_raw['artist_song_count'] = song_count_input.value

  # Genre
  genre_dropdown = dropdown_inputs['genre_input']
  genre_name = genre_dropdown.value
  user_vals['genre_' + genre_name] = 1.0
  input_vals_raw['genre_avg_popularity'] = genre_to_avg_pop.get(genre_name, 0.0)
  input_vals_raw['genre_song_count'] = genre_to_song_count.get(genre_name, 0.0)

  # Year
  year_dropdown = dropdown_inputs['year_input']
  year_val = year_dropdown.value
  user_vals['year_' + year_val] = 1.0
  input_vals_raw['year_avg_popularity'] = year_to_avg_pop.get(int(year_val), 0.0)
  input_vals_raw['year_song_count'] = year_to_song_count.get(int(year_val), 0.0)


  # Key, mode, time_sig
  key_dropdown = dropdown_inputs['key_input']
  mode_dropdown = dropdown_inputs['mode_input']
  time_sig_dropdown = dropdown_inputs['time_sig_input']

  user_vals['keyOf' + key_dropdown.value] = 1.0
  user_vals[mode_dropdown.value] = 1.0
  user_vals['time_signature_' + time_sig_dropdown.value.replace('/','')] = 1.0


  """
  print(f"loudness = {input_vals_raw['loudness']}")
  print(f"tempo = {input_vals_raw['tempo']}")
  print(f"duration_mins = {input_vals_raw['duration_mins']}")
  print(f"artist_avg_popularity = {input_vals_raw['artist_avg_popularity']}")
  print(f"artist_song_count = {input_vals_raw['artist_song_count']}")
  print(f"\ngenre = {genre_dropdown.value}")
  print(f"genre_avg_popularity = {input_vals_raw['genre_avg_popularity']}")
  print(f"genre_song_count = {input_vals_raw['genre_song_count']}")
  print(f"\nyear = {year_dropdown.value}")
  print(f"year_avg_popularity = {input_vals_raw['year_avg_popularity']}")
  print(f"year_song_count = {input_vals_raw['year_song_count']}")
  print(f"keyOf = {key_dropdown.value}")
  print(f"mode = {mode_dropdown.value}")
  print(f"time_signature = {time_sig_dropdown.value}")
  print(f"user_vals = {user_vals}")
  """

  # Apply manual scaling from Section 1

  input_vals_raw['loudness'] = np.clip(input_vals_raw['loudness'], -60, 0)
  input_vals_raw['loudness'] = (input_vals_raw['loudness'] + 60) / 60


  input_vals_raw['tempo'] = np.clip(input_vals_raw['tempo'], 50, 220)
  input_vals_raw['tempo'] = (input_vals_raw['tempo'] - 50) / 170


  input_vals_raw['duration_mins'] = np.clip(input_vals_raw['duration_mins'], 0, 20)
  input_vals_raw['duration_mins'] = input_vals_raw['duration_mins'] / 20

  input_vals_raw['artist_avg_popularity'] = np.clip(input_vals_raw['artist_avg_popularity'], 0, 100) / 100

  input_vals_raw['artist_song_count'] = np.clip(input_vals_raw['artist_song_count'], 0, 3000) / 3000

  input_vals_raw['genre_avg_popularity'] = np.clip(input_vals_raw['genre_avg_popularity'], 0, 100) / 100

  input_vals_raw['genre_song_count'] = np.clip(input_vals_raw['genre_song_count'], 0, 30000) / 30000

  input_vals_raw['year_avg_popularity'] = np.clip(input_vals_raw['year_avg_popularity'], 0, 100) / 100

  input_vals_raw['year_song_count'] = np.clip(input_vals_raw['year_song_count'], 0, 50000) / 50000


  # Assemble input for model

  #print(user_vals)
  #print(input_vals_raw)

  input_df = pd.DataFrame([{**input_vals_raw, **user_vals}])
  input_df = input_df.reindex(columns=X_train.columns, fill_value=0.0)

  prediction = float(model.predict(input_df,verbose=False)[0][0])

  display_pred = input_df.loc[:, input_df.ne(0).any(axis=0)]
  display_pred.insert(0, 'popularity', prediction)
  display(display_pred)
  return prediction

**[END]**

.

.

.

.

.

.

.

.

.

.


#**APPLICATION**

###LOADING

ensure to load spotidata.csv and model.keras into the environment, and then run Section 1, Section 5 and APPLICATION in order to use the UI.

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model('model.keras')

In [ ]:
X = train_df.drop(labels='popularity', axis=1)

y = train_df['popularity']

#Train and Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, shuffle=True, random_state=0)

###UI

In [ ]:
predict_button.on_click(on_predict)
display(ui)